### Construct block-level DVAP data for Oregon, using population data from DC20 and citizenship percentages computed from ACS22.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', None)

import numpy as np
import json
import networkx as nx
import math
import random

from collections import Counter

import maup
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
maup.progress.enabled = True


from gerrychain import Graph, GeographicPartition, Partition, updaters, Election
from gerrychain.updaters import Tally, cut_edges

from gerrytools.mgrp import *
from gerrytools.data import census20
from gerrytools.data.acs import acs5
from us.states import OR

from networkx.algorithms import tree

from tqdm import tqdm

### Block-level VAP data from Census P3 and P4 tables:

In [2]:
state = OR
unit = "block"
index = "GEOID20"

p3 = census20(state, "P3", geometry=unit)
p4 = census20(state, "P4", geometry=unit)

census_vap_df = p3.merge(p4, on="GEOID20")

In [3]:
list(census_vap_df.columns)

['GEOID20',
 'VAP20_x',
 'WHITEVAP20',
 'BLACKVAP20',
 'AMINVAP20',
 'ASIANVAP20',
 'NHPIVAP20',
 'OTHVAP20',
 'WHITEBLACKVAP20',
 'WHITEAMINVAP20',
 'WHITEASIANVAP20',
 'WHITENHPIVAP20',
 'WHITEOTHVAP20',
 'BLACKAMINVAP20',
 'BLACKASIANVAP20',
 'BLACKNHPIVAP20',
 'BLACKOTHVAP20',
 'AMINASIANVAP20',
 'AMINNHPIVAP20',
 'AMINOTHVAP20',
 'ASIANNHPIVAP20',
 'ASIANOTHVAP20',
 'NHPIOTHVAP20',
 'WHITEBLACKAMINVAP20',
 'WHITEBLACKASIANVAP20',
 'WHITEBLACKNHPIVAP20',
 'WHITEBLACKOTHVAP20',
 'WHITEAMINASIANVAP20',
 'WHITEAMINNHPIVAP20',
 'WHITEAMINOTHVAP20',
 'WHITEASIANNHPIVAP20',
 'WHITEASIANOTHVAP20',
 'WHITENHPIOTHVAP20',
 'BLACKAMINASIANVAP20',
 'BLACKAMINNHPIVAP20',
 'BLACKAMINOTHVAP20',
 'BLACKASIANNHPIVAP20',
 'BLACKASIANOTHVAP20',
 'BLACKNHPIOTHVAP20',
 'AMINASIANNHPIVAP20',
 'AMINASIANOTHVAP20',
 'AMINNHPIOTHVAP20',
 'ASIANNHPIOTHVAP20',
 'WHITEBLACKAMINASIANVAP20',
 'WHITEBLACKAMINNHPIVAP20',
 'WHITEBLACKAMINOTHVAP20',
 'WHITEBLACKASIANNHPIVAP20',
 'WHITEBLACKASIANOTHVAP20',
 'WHITEBL

### Identify/create Hispanic/Non-Hispanic columns from the P4 columns:

In [4]:
def is_non_hispanic(col):
    return col.startswith("NH") and not col.startswith("NHPI")

In [5]:
non_hispanic_cols = [col for col in census_vap_df.columns if is_non_hispanic(col)]

hispanic_cols = []
for col in non_hispanic_cols:
    census_vap_df[col[1:]] = census_vap_df[col[2:]] - census_vap_df[col]
    hispanic_cols.append(col[1:])
all_cols = hispanic_cols + non_hispanic_cols

In [6]:
list(census_vap_df.columns)

['GEOID20',
 'VAP20_x',
 'WHITEVAP20',
 'BLACKVAP20',
 'AMINVAP20',
 'ASIANVAP20',
 'NHPIVAP20',
 'OTHVAP20',
 'WHITEBLACKVAP20',
 'WHITEAMINVAP20',
 'WHITEASIANVAP20',
 'WHITENHPIVAP20',
 'WHITEOTHVAP20',
 'BLACKAMINVAP20',
 'BLACKASIANVAP20',
 'BLACKNHPIVAP20',
 'BLACKOTHVAP20',
 'AMINASIANVAP20',
 'AMINNHPIVAP20',
 'AMINOTHVAP20',
 'ASIANNHPIVAP20',
 'ASIANOTHVAP20',
 'NHPIOTHVAP20',
 'WHITEBLACKAMINVAP20',
 'WHITEBLACKASIANVAP20',
 'WHITEBLACKNHPIVAP20',
 'WHITEBLACKOTHVAP20',
 'WHITEAMINASIANVAP20',
 'WHITEAMINNHPIVAP20',
 'WHITEAMINOTHVAP20',
 'WHITEASIANNHPIVAP20',
 'WHITEASIANOTHVAP20',
 'WHITENHPIOTHVAP20',
 'BLACKAMINASIANVAP20',
 'BLACKAMINNHPIVAP20',
 'BLACKAMINOTHVAP20',
 'BLACKASIANNHPIVAP20',
 'BLACKASIANOTHVAP20',
 'BLACKNHPIOTHVAP20',
 'AMINASIANNHPIVAP20',
 'AMINASIANOTHVAP20',
 'AMINNHPIOTHVAP20',
 'ASIANNHPIOTHVAP20',
 'WHITEBLACKAMINASIANVAP20',
 'WHITEBLACKAMINNHPIVAP20',
 'WHITEBLACKAMINOTHVAP20',
 'WHITEBLACKASIANNHPIVAP20',
 'WHITEBLACKASIANOTHVAP20',
 'WHITEBL

### Note that we now have two VAP20 columns; check that they are equal and remove one.

In [7]:
census_vap_df[census_vap_df["VAP20_x"] != census_vap_df["VAP20_y"]]

,GEOID20,VAP20_x,WHITEVAP20,BLACKVAP20,AMINVAP20,ASIANVAP20,NHPIVAP20,OTHVAP20,WHITEBLACKVAP20,WHITEAMINVAP20,WHITEASIANVAP20,WHITENHPIVAP20,WHITEOTHVAP20,BLACKAMINVAP20,BLACKASIANVAP20,BLACKNHPIVAP20,BLACKOTHVAP20,AMINASIANVAP20,AMINNHPIVAP20,AMINOTHVAP20,ASIANNHPIVAP20,ASIANOTHVAP20,NHPIOTHVAP20,WHITEBLACKAMINVAP20,WHITEBLACKASIANVAP20,WHITEBLACKNHPIVAP20,WHITEBLACKOTHVAP20,WHITEAMINASIANVAP20,WHITEAMINNHPIVAP20,WHITEAMINOTHVAP20,WHITEASIANNHPIVAP20,WHITEASIANOTHVAP20,WHITENHPIOTHVAP20,BLACKAMINASIANVAP20,BLACKAMINNHPIVAP20,BLACKAMINOTHVAP20,BLACKASIANNHPIVAP20,BLACKASIANOTHVAP20,BLACKNHPIOTHVAP20,AMINASIANNHPIVAP20,AMINASIANOTHVAP20,AMINNHPIOTHVAP20,ASIANNHPIOTHVAP20,WHITEBLACKAMINASIANVAP20,WHITEBLACKAMINNHPIVAP20,WHITEBLACKAMINOTHVAP20,WHITEBLACKASIANNHPIVAP20,WHITEBLACKASIANOTHVAP20,WHITEBLACKNHPIOTHVAP20,WHITEAMINASIANNHPIVAP20,WHITEAMINASIANOTHVAP20,WHITEAMINNHPIOTHVAP20,WHITEASIANNHPIOTHVAP20,BLACKAMINASIANNHPIVAP20,BLACKAMINASIANOTHVAP20,BLACKAMINNHPIOTHVAP20,BLACKASIANNHPIOTHVAP20,AMINASIANNHPIOTHVAP20,WHITEBLACKAMINASIANNHPIVAP20,WHITEBLACKAMINASIANOTHVAP20,WHITEBLACKAMINNHPIOTHVAP20,WHITEBLACKASIANNHPIOTHVAP20,WHITEAMINASIANNHPIOTHVAP20,BLACKAMINASIANNHPIOTHVAP20,WHITEBLACKAMINASIANNHPIOTHVAP20,VAP20_y,HVAP20,NHWHITEVAP20,NHBLACKVAP20,NHAMINVAP20,NHASIANVAP20,NHNHPIVAP20,NHOTHVAP20,NHWHITEBLACKVAP20,NHWHITEAMINVAP20,NHWHITEASIANVAP20,NHWHITENHPIVAP20,NHWHITEOTHVAP20,NHBLACKAMINVAP20,NHBLACKASIANVAP20,NHBLACKNHPIVAP20,NHBLACKOTHVAP20,NHAMINASIANVAP20,NHAMINNHPIVAP20,NHAMINOTHVAP20,NHASIANNHPIVAP20,NHASIANOTHVAP20,NHNHPIOTHVAP20,NHWHITEBLACKAMINVAP20,NHWHITEBLACKASIANVAP20,NHWHITEBLACKNHPIVAP20,NHWHITEBLACKOTHVAP20,NHWHITEAMINASIANVAP20,NHWHITEAMINNHPIVAP20,NHWHITEAMINOTHVAP20,NHWHITEASIANNHPIVAP20,NHWHITEASIANOTHVAP20,NHWHITENHPIOTHVAP20,NHBLACKAMINASIANVAP20,NHBLACKAMINNHPIVAP20,NHBLACKAMINOTHVAP20,NHBLACKASIANNHPIVAP20,NHBLACKASIANOTHVAP20,NHBLACKNHPIOTHVAP20,NHAMINASIANNHPIVAP20,NHAMINASIANOTHVAP20,NHAMINNHPIOTHVAP20,NHASIANNHPIOTHVAP20,NHWHITEBLACKAMINASIANVAP20,NHWHITEBLACKAMINNHPIVAP20,NHWHITEBLACKAMINOTHVAP20,NHWHITEBLACKASIANNHPIVAP20,NHWHITEBLACKASIANOTHVAP20,NHWHITEBLACKNHPIOTHVAP20,NHWHITEAMINASIANNHPIVAP20,NHWHITEAMINASIANOTHVAP20,NHWHITEAMINNHPIOTHVAP20,NHWHITEASIANNHPIOTHVAP20,NHBLACKAMINASIANNHPIVAP20,NHBLACKAMINASIANOTHVAP20,NHBLACKAMINNHPIOTHVAP20,NHBLACKASIANNHPIOTHVAP20,NHAMINASIANNHPIOTHVAP20,NHWHITEBLACKAMINASIANNHPIVAP20,NHWHITEBLACKAMINASIANOTHVAP20,NHWHITEBLACKAMINNHPIOTHVAP20,NHWHITEBLACKASIANNHPIOTHVAP20,NHWHITEAMINASIANNHPIOTHVAP20,NHBLACKAMINASIANNHPIOTHVAP20,NHWHITEBLACKAMINASIANNHPIOTHVAP20,HWHITEVAP20,HBLACKVAP20,HAMINVAP20,HASIANVAP20,HNHPIVAP20,HOTHVAP20,HWHITEBLACKVAP20,HWHITEAMINVAP20,HWHITEASIANVAP20,HWHITENHPIVAP20,HWHITEOTHVAP20,HBLACKAMINVAP20,HBLACKASIANVAP20,HBLACKNHPIVAP20,HBLACKOTHVAP20,HAMINASIANVAP20,HAMINNHPIVAP20,HAMINOTHVAP20,HASIANNHPIVAP20,HASIANOTHVAP20,HNHPIOTHVAP20,HWHITEBLACKAMINVAP20,HWHITEBLACKASIANVAP20,HWHITEBLACKNHPIVAP20,HWHITEBLACKOTHVAP20,HWHITEAMINASIANVAP20,HWHITEAMINNHPIVAP20,HWHITEAMINOTHVAP20,HWHITEASIANNHPIVAP20,HWHITEASIANOTHVAP20,HWHITENHPIOTHVAP20,HBLACKAMINASIANVAP20,HBLACKAMINNHPIVAP20,HBLACKAMINOTHVAP20,HBLACKASIANNHPIVAP20,HBLACKASIANOTHVAP20,HBLACKNHPIOTHVAP20,HAMINASIANNHPIVAP20,HAMINASIANOTHVAP20,HAMINNHPIOTHVAP20,HASIANNHPIOTHVAP20,HWHITEBLACKAMINASIANVAP20,HWHITEBLACKAMINNHPIVAP20,HWHITEBLACKAMINOTHVAP20,HWHITEBLACKASIANNHPIVAP20,HWHITEBLACKASIANOTHVAP20,HWHITEBLACKNHPIOTHVAP20,HWHITEAMINASIANNHPIVAP20,HWHITEAMINASIANOTHVAP20,HWHITEAMINNHPIOTHVAP20,HWHITEASIANNHPIOTHVAP20,HBLACKAMINASIANNHPIVAP20,HBLACKAMINASIANOTHVAP20,HBLACKAMINNHPIOTHVAP20,HBLACKASIANNHPIOTHVAP20,HAMINASIANNHPIOTHVAP20,HWHITEBLACKAMINASIANNHPIVAP20,HWHITEBLACKAMINASIANOTHVAP20,HWHITEBLACKAMINNHPIOTHVAP20,HWHITEBLACKASIANNHPIOTHVAP20,HWHITEAMINASIANNHPIOTHVAP20,HBLACKAMINASIANNHPIOTHVAP20,HWHITEBLACKAMINASIANNHPIOTHVAP20


In [8]:
census_vap_df = census_vap_df.drop(columns = ['VAP20_y'])
census_vap_df = census_vap_df.rename(columns = {'VAP20_x': 'VAP20'})

### Do the newly created Hispanic columns sum to HVAP?

In [9]:
census_vap_df[census_vap_df[hispanic_cols].sum(axis=1) != census_vap_df["HVAP20"]]

,GEOID20,VAP20,WHITEVAP20,BLACKVAP20,AMINVAP20,ASIANVAP20,NHPIVAP20,OTHVAP20,WHITEBLACKVAP20,WHITEAMINVAP20,WHITEASIANVAP20,WHITENHPIVAP20,WHITEOTHVAP20,BLACKAMINVAP20,BLACKASIANVAP20,BLACKNHPIVAP20,BLACKOTHVAP20,AMINASIANVAP20,AMINNHPIVAP20,AMINOTHVAP20,ASIANNHPIVAP20,ASIANOTHVAP20,NHPIOTHVAP20,WHITEBLACKAMINVAP20,WHITEBLACKASIANVAP20,WHITEBLACKNHPIVAP20,WHITEBLACKOTHVAP20,WHITEAMINASIANVAP20,WHITEAMINNHPIVAP20,WHITEAMINOTHVAP20,WHITEASIANNHPIVAP20,WHITEASIANOTHVAP20,WHITENHPIOTHVAP20,BLACKAMINASIANVAP20,BLACKAMINNHPIVAP20,BLACKAMINOTHVAP20,BLACKASIANNHPIVAP20,BLACKASIANOTHVAP20,BLACKNHPIOTHVAP20,AMINASIANNHPIVAP20,AMINASIANOTHVAP20,AMINNHPIOTHVAP20,ASIANNHPIOTHVAP20,WHITEBLACKAMINASIANVAP20,WHITEBLACKAMINNHPIVAP20,WHITEBLACKAMINOTHVAP20,WHITEBLACKASIANNHPIVAP20,WHITEBLACKASIANOTHVAP20,WHITEBLACKNHPIOTHVAP20,WHITEAMINASIANNHPIVAP20,WHITEAMINASIANOTHVAP20,WHITEAMINNHPIOTHVAP20,WHITEASIANNHPIOTHVAP20,BLACKAMINASIANNHPIVAP20,BLACKAMINASIANOTHVAP20,BLACKAMINNHPIOTHVAP20,BLACKASIANNHPIOTHVAP20,AMINASIANNHPIOTHVAP20,WHITEBLACKAMINASIANNHPIVAP20,WHITEBLACKAMINASIANOTHVAP20,WHITEBLACKAMINNHPIOTHVAP20,WHITEBLACKASIANNHPIOTHVAP20,WHITEAMINASIANNHPIOTHVAP20,BLACKAMINASIANNHPIOTHVAP20,WHITEBLACKAMINASIANNHPIOTHVAP20,HVAP20,NHWHITEVAP20,NHBLACKVAP20,NHAMINVAP20,NHASIANVAP20,NHNHPIVAP20,NHOTHVAP20,NHWHITEBLACKVAP20,NHWHITEAMINVAP20,NHWHITEASIANVAP20,NHWHITENHPIVAP20,NHWHITEOTHVAP20,NHBLACKAMINVAP20,NHBLACKASIANVAP20,NHBLACKNHPIVAP20,NHBLACKOTHVAP20,NHAMINASIANVAP20,NHAMINNHPIVAP20,NHAMINOTHVAP20,NHASIANNHPIVAP20,NHASIANOTHVAP20,NHNHPIOTHVAP20,NHWHITEBLACKAMINVAP20,NHWHITEBLACKASIANVAP20,NHWHITEBLACKNHPIVAP20,NHWHITEBLACKOTHVAP20,NHWHITEAMINASIANVAP20,NHWHITEAMINNHPIVAP20,NHWHITEAMINOTHVAP20,NHWHITEASIANNHPIVAP20,NHWHITEASIANOTHVAP20,NHWHITENHPIOTHVAP20,NHBLACKAMINASIANVAP20,NHBLACKAMINNHPIVAP20,NHBLACKAMINOTHVAP20,NHBLACKASIANNHPIVAP20,NHBLACKASIANOTHVAP20,NHBLACKNHPIOTHVAP20,NHAMINASIANNHPIVAP20,NHAMINASIANOTHVAP20,NHAMINNHPIOTHVAP20,NHASIANNHPIOTHVAP20,NHWHITEBLACKAMINASIANVAP20,NHWHITEBLACKAMINNHPIVAP20,NHWHITEBLACKAMINOTHVAP20,NHWHITEBLACKASIANNHPIVAP20,NHWHITEBLACKASIANOTHVAP20,NHWHITEBLACKNHPIOTHVAP20,NHWHITEAMINASIANNHPIVAP20,NHWHITEAMINASIANOTHVAP20,NHWHITEAMINNHPIOTHVAP20,NHWHITEASIANNHPIOTHVAP20,NHBLACKAMINASIANNHPIVAP20,NHBLACKAMINASIANOTHVAP20,NHBLACKAMINNHPIOTHVAP20,NHBLACKASIANNHPIOTHVAP20,NHAMINASIANNHPIOTHVAP20,NHWHITEBLACKAMINASIANNHPIVAP20,NHWHITEBLACKAMINASIANOTHVAP20,NHWHITEBLACKAMINNHPIOTHVAP20,NHWHITEBLACKASIANNHPIOTHVAP20,NHWHITEAMINASIANNHPIOTHVAP20,NHBLACKAMINASIANNHPIOTHVAP20,NHWHITEBLACKAMINASIANNHPIOTHVAP20,HWHITEVAP20,HBLACKVAP20,HAMINVAP20,HASIANVAP20,HNHPIVAP20,HOTHVAP20,HWHITEBLACKVAP20,HWHITEAMINVAP20,HWHITEASIANVAP20,HWHITENHPIVAP20,HWHITEOTHVAP20,HBLACKAMINVAP20,HBLACKASIANVAP20,HBLACKNHPIVAP20,HBLACKOTHVAP20,HAMINASIANVAP20,HAMINNHPIVAP20,HAMINOTHVAP20,HASIANNHPIVAP20,HASIANOTHVAP20,HNHPIOTHVAP20,HWHITEBLACKAMINVAP20,HWHITEBLACKASIANVAP20,HWHITEBLACKNHPIVAP20,HWHITEBLACKOTHVAP20,HWHITEAMINASIANVAP20,HWHITEAMINNHPIVAP20,HWHITEAMINOTHVAP20,HWHITEASIANNHPIVAP20,HWHITEASIANOTHVAP20,HWHITENHPIOTHVAP20,HBLACKAMINASIANVAP20,HBLACKAMINNHPIVAP20,HBLACKAMINOTHVAP20,HBLACKASIANNHPIVAP20,HBLACKASIANOTHVAP20,HBLACKNHPIOTHVAP20,HAMINASIANNHPIVAP20,HAMINASIANOTHVAP20,HAMINNHPIOTHVAP20,HASIANNHPIOTHVAP20,HWHITEBLACKAMINASIANVAP20,HWHITEBLACKAMINNHPIVAP20,HWHITEBLACKAMINOTHVAP20,HWHITEBLACKASIANNHPIVAP20,HWHITEBLACKASIANOTHVAP20,HWHITEBLACKNHPIOTHVAP20,HWHITEAMINASIANNHPIVAP20,HWHITEAMINASIANOTHVAP20,HWHITEAMINNHPIOTHVAP20,HWHITEASIANNHPIOTHVAP20,HBLACKAMINASIANNHPIVAP20,HBLACKAMINASIANOTHVAP20,HBLACKAMINNHPIOTHVAP20,HBLACKASIANNHPIOTHVAP20,HAMINASIANNHPIOTHVAP20,HWHITEBLACKAMINASIANNHPIVAP20,HWHITEBLACKAMINASIANOTHVAP20,HWHITEBLACKAMINNHPIOTHVAP20,HWHITEBLACKASIANNHPIOTHVAP20,HWHITEAMINASIANNHPIOTHVAP20,HBLACKAMINASIANNHPIOTHVAP20,HWHITEBLACKAMINASIANNHPIOTHVAP20


### Excellent!

### Assign decennial census columns to litigation categories; we decided to use the order
### HISP > AAPI > BLACK > AMIN > NHWHITE+OTHER

In [10]:
litcat = {}
for col in all_cols:
    if col.startswith("H"):
        litcat[col] = "LIT_HISP"
    elif ("ASIAN" in col or "NHPI" in col):
        litcat[col] = "LIT_AAPI"
    elif "BLACK" in col:
        litcat[col] = "LIT_BLACK"
    elif "AMIN" in col:
        litcat[col] = "LIT_AMIN"
    else: # includes NHWHITE + NHOTHER
        litcat[col] = "LIT_WHITE"

### Create new columns in census_vap_df for litigation categories, including a blanket "POC" category:

In [11]:
census_vap_df['L_BVAP'] = census_vap_df[[col for col in all_cols if litcat[col] == 'LIT_BLACK']].sum(axis=1)
census_vap_df['L_HVAP'] = census_vap_df[[col for col in all_cols if litcat[col] == 'LIT_HISP']].sum(axis=1)
census_vap_df['L_AAPIVAP'] = census_vap_df[[col for col in all_cols if litcat[col] == 'LIT_AAPI']].sum(axis=1)
census_vap_df['L_AMINVAP'] = census_vap_df[[col for col in all_cols if litcat[col] == 'LIT_AMIN']].sum(axis=1)
#census_vap_df['L_OTHVAP'] = census_vap_df[[col for col in all_cols if litcat[col] == 'LIT_OTH']].sum(axis=1)
census_vap_df['L_WVAP'] = census_vap_df[[col for col in all_cols if litcat[col] == 'LIT_WHITE']].sum(axis=1)
#census_vap_df['L_POC'] = census_vap_df['VAP20'] - census_vap_df['L_WVAP']

### Check that the litigation categories sum to total VAP:

In [12]:
test = census_vap_df['L_BVAP'] + census_vap_df['L_HVAP'] + census_vap_df['L_AAPIVAP'] + census_vap_df['L_AMINVAP'] + census_vap_df['L_WVAP'] - census_vap_df['VAP20'] 

[i for i in test.index if test[i] != 0]

[]

### We also need a map of blocks to tracts:

In [13]:
census_vap_df

,GEOID20,VAP20,WHITEVAP20,BLACKVAP20,AMINVAP20,ASIANVAP20,NHPIVAP20,OTHVAP20,WHITEBLACKVAP20,WHITEAMINVAP20,WHITEASIANVAP20,WHITENHPIVAP20,WHITEOTHVAP20,BLACKAMINVAP20,BLACKASIANVAP20,BLACKNHPIVAP20,BLACKOTHVAP20,AMINASIANVAP20,AMINNHPIVAP20,AMINOTHVAP20,ASIANNHPIVAP20,ASIANOTHVAP20,NHPIOTHVAP20,WHITEBLACKAMINVAP20,WHITEBLACKASIANVAP20,WHITEBLACKNHPIVAP20,WHITEBLACKOTHVAP20,WHITEAMINASIANVAP20,WHITEAMINNHPIVAP20,WHITEAMINOTHVAP20,WHITEASIANNHPIVAP20,WHITEASIANOTHVAP20,WHITENHPIOTHVAP20,BLACKAMINASIANVAP20,BLACKAMINNHPIVAP20,BLACKAMINOTHVAP20,BLACKASIANNHPIVAP20,BLACKASIANOTHVAP20,BLACKNHPIOTHVAP20,AMINASIANNHPIVAP20,AMINASIANOTHVAP20,AMINNHPIOTHVAP20,ASIANNHPIOTHVAP20,WHITEBLACKAMINASIANVAP20,WHITEBLACKAMINNHPIVAP20,WHITEBLACKAMINOTHVAP20,WHITEBLACKASIANNHPIVAP20,WHITEBLACKASIANOTHVAP20,WHITEBLACKNHPIOTHVAP20,WHITEAMINASIANNHPIVAP20,WHITEAMINASIANOTHVAP20,WHITEAMINNHPIOTHVAP20,WHITEASIANNHPIOTHVAP20,BLACKAMINASIANNHPIVAP20,BLACKAMINASIANOTHVAP20,BLACKAMINNHPIOTHVAP20,BLACKASIANNHPIOTHVAP20,AMINASIANNHPIOTHVAP20,WHITEBLACKAMINASIANNHPIVAP20,WHITEBLACKAMINASIANOTHVAP20,WHITEBLACKAMINNHPIOTHVAP20,WHITEBLACKASIANNHPIOTHVAP20,WHITEAMINASIANNHPIOTHVAP20,BLACKAMINASIANNHPIOTHVAP20,WHITEBLACKAMINASIANNHPIOTHVAP20,HVAP20,NHWHITEVAP20,NHBLACKVAP20,NHAMINVAP20,NHASIANVAP20,NHNHPIVAP20,NHOTHVAP20,NHWHITEBLACKVAP20,NHWHITEAMINVAP20,NHWHITEASIANVAP20,NHWHITENHPIVAP20,NHWHITEOTHVAP20,NHBLACKAMINVAP20,NHBLACKASIANVAP20,NHBLACKNHPIVAP20,NHBLACKOTHVAP20,NHAMINASIANVAP20,NHAMINNHPIVAP20,NHAMINOTHVAP20,NHASIANNHPIVAP20,NHASIANOTHVAP20,NHNHPIOTHVAP20,NHWHITEBLACKAMINVAP20,NHWHITEBLACKASIANVAP20,NHWHITEBLACKNHPIVAP20,NHWHITEBLACKOTHVAP20,NHWHITEAMINASIANVAP20,NHWHITEAMINNHPIVAP20,NHWHITEAMINOTHVAP20,NHWHITEASIANNHPIVAP20,NHWHITEASIANOTHVAP20,NHWHITENHPIOTHVAP20,NHBLACKAMINASIANVAP20,NHBLACKAMINNHPIVAP20,NHBLACKAMINOTHVAP20,NHBLACKASIANNHPIVAP20,NHBLACKASIANOTHVAP20,NHBLACKNHPIOTHVAP20,NHAMINASIANNHPIVAP20,NHAMINASIANOTHVAP20,NHAMINNHPIOTHVAP20,NHASIANNHPIOTHVAP20,NHWHITEBLACKAMINASIANVAP20,NHWHITEBLACKAMINNHPIVAP20,NHWHITEBLACKAMINOTHVAP20,NHWHITEBLACKASIANNHPIVAP20,NHWHITEBLACKASIANOTHVAP20,NHWHITEBLACKNHPIOTHVAP20,NHWHITEAMINASIANNHPIVAP20,NHWHITEAMINASIANOTHVAP20,NHWHITEAMINNHPIOTHVAP20,NHWHITEASIANNHPIOTHVAP20,NHBLACKAMINASIANNHPIVAP20,NHBLACKAMINASIANOTHVAP20,NHBLACKAMINNHPIOTHVAP20,NHBLACKASIANNHPIOTHVAP20,NHAMINASIANNHPIOTHVAP20,NHWHITEBLACKAMINASIANNHPIVAP20,NHWHITEBLACKAMINASIANOTHVAP20,NHWHITEBLACKAMINNHPIOTHVAP20,NHWHITEBLACKASIANNHPIOTHVAP20,NHWHITEAMINASIANNHPIOTHVAP20,NHBLACKAMINASIANNHPIOTHVAP20,NHWHITEBLACKAMINASIANNHPIOTHVAP20,HWHITEVAP20,HBLACKVAP20,HAMINVAP20,HASIANVAP20,HNHPIVAP20,HOTHVAP20,HWHITEBLACKVAP20,HWHITEAMINVAP20,HWHITEASIANVAP20,HWHITENHPIVAP20,HWHITEOTHVAP20,HBLACKAMINVAP20,HBLACKASIANVAP20,HBLACKNHPIVAP20,HBLACKOTHVAP20,HAMINASIANVAP20,HAMINNHPIVAP20,HAMINOTHVAP20,HASIANNHPIVAP20,HASIANOTHVAP20,HNHPIOTHVAP20,HWHITEBLACKAMINVAP20,HWHITEBLACKASIANVAP20,HWHITEBLACKNHPIVAP20,HWHITEBLACKOTHVAP20,HWHITEAMINASIANVAP20,HWHITEAMINNHPIVAP20,HWHITEAMINOTHVAP20,HWHITEASIANNHPIVAP20,HWHITEASIANOTHVAP20,HWHITENHPIOTHVAP20,HBLACKAMINASIANVAP20,HBLACKAMINNHPIVAP20,HBLACKAMINOTHVAP20,HBLACKASIANNHPIVAP20,HBLACKASIANOTHVAP20,HBLACKNHPIOTHVAP20,HAMINASIANNHPIVAP20,HAMINASIANOTHVAP20,HAMINNHPIOTHVAP20,HASIANNHPIOTHVAP20,HWHITEBLACKAMINASIANVAP20,HWHITEBLACKAMINNHPIVAP20,HWHITEBLACKAMINOTHVAP20,HWHITEBLACKASIANNHPIVAP20,HWHITEBLACKASIANOTHVAP20,HWHITEBLACKNHPIOTHVAP20,HWHITEAMINASIANNHPIVAP20,HWHITEAMINASIANOTHVAP20,HWHITEAMINNHPIOTHVAP20,HWHITEASIANNHPIOTHVAP20,HBLACKAMINASIANNHPIVAP20,HBLACKAMINASIANOTHVAP20,HBLACKAMINNHPIOTHVAP20,HBLACKASIANNHPIOTHVAP20,HAMINASIANNHPIOTHVAP20,HWHITEBLACKAMINASIANNHPIVAP20,HWHITEBLACKAMINASIANOTHVAP20,HWHITEBLACKAMINNHPIOTHVAP20,HWHITEBLACKASIANNHPIOTHVAP20,HWHITEAMINASIANNHPIOTHVAP20,HBLACKAMINASIANNHPIOTHVAP20,HWHITEBLACKAMINASIANNHPIOTHVAP20,L_BVAP,L_HVAP,L_AAPIVAP,L_AMINVAP,L_WVAP
0,410670335003015,3,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [14]:
census_vap_df.insert(1, 'TRACT20', None)

In [15]:
for i in tqdm(census_vap_df.index):
    census_vap_df.loc[i, 'TRACT20'] = census_vap_df['GEOID20'][i][:-4]

100%|████████████████████████████████| 130807/130807 [00:06<00:00, 20815.49it/s]


In [16]:
census_vap_df

,GEOID20,TRACT20,VAP20,WHITEVAP20,BLACKVAP20,AMINVAP20,ASIANVAP20,NHPIVAP20,OTHVAP20,WHITEBLACKVAP20,WHITEAMINVAP20,WHITEASIANVAP20,WHITENHPIVAP20,WHITEOTHVAP20,BLACKAMINVAP20,BLACKASIANVAP20,BLACKNHPIVAP20,BLACKOTHVAP20,AMINASIANVAP20,AMINNHPIVAP20,AMINOTHVAP20,ASIANNHPIVAP20,ASIANOTHVAP20,NHPIOTHVAP20,WHITEBLACKAMINVAP20,WHITEBLACKASIANVAP20,WHITEBLACKNHPIVAP20,WHITEBLACKOTHVAP20,WHITEAMINASIANVAP20,WHITEAMINNHPIVAP20,WHITEAMINOTHVAP20,WHITEASIANNHPIVAP20,WHITEASIANOTHVAP20,WHITENHPIOTHVAP20,BLACKAMINASIANVAP20,BLACKAMINNHPIVAP20,BLACKAMINOTHVAP20,BLACKASIANNHPIVAP20,BLACKASIANOTHVAP20,BLACKNHPIOTHVAP20,AMINASIANNHPIVAP20,AMINASIANOTHVAP20,AMINNHPIOTHVAP20,ASIANNHPIOTHVAP20,WHITEBLACKAMINASIANVAP20,WHITEBLACKAMINNHPIVAP20,WHITEBLACKAMINOTHVAP20,WHITEBLACKASIANNHPIVAP20,WHITEBLACKASIANOTHVAP20,WHITEBLACKNHPIOTHVAP20,WHITEAMINASIANNHPIVAP20,WHITEAMINASIANOTHVAP20,WHITEAMINNHPIOTHVAP20,WHITEASIANNHPIOTHVAP20,BLACKAMINASIANNHPIVAP20,BLACKAMINASIANOTHVAP20,BLACKAMINNHPIOTHVAP20,BLACKASIANNHPIOTHVAP20,AMINASIANNHPIOTHVAP20,WHITEBLACKAMINASIANNHPIVAP20,WHITEBLACKAMINASIANOTHVAP20,WHITEBLACKAMINNHPIOTHVAP20,WHITEBLACKASIANNHPIOTHVAP20,WHITEAMINASIANNHPIOTHVAP20,BLACKAMINASIANNHPIOTHVAP20,WHITEBLACKAMINASIANNHPIOTHVAP20,HVAP20,NHWHITEVAP20,NHBLACKVAP20,NHAMINVAP20,NHASIANVAP20,NHNHPIVAP20,NHOTHVAP20,NHWHITEBLACKVAP20,NHWHITEAMINVAP20,NHWHITEASIANVAP20,NHWHITENHPIVAP20,NHWHITEOTHVAP20,NHBLACKAMINVAP20,NHBLACKASIANVAP20,NHBLACKNHPIVAP20,NHBLACKOTHVAP20,NHAMINASIANVAP20,NHAMINNHPIVAP20,NHAMINOTHVAP20,NHASIANNHPIVAP20,NHASIANOTHVAP20,NHNHPIOTHVAP20,NHWHITEBLACKAMINVAP20,NHWHITEBLACKASIANVAP20,NHWHITEBLACKNHPIVAP20,NHWHITEBLACKOTHVAP20,NHWHITEAMINASIANVAP20,NHWHITEAMINNHPIVAP20,NHWHITEAMINOTHVAP20,NHWHITEASIANNHPIVAP20,NHWHITEASIANOTHVAP20,NHWHITENHPIOTHVAP20,NHBLACKAMINASIANVAP20,NHBLACKAMINNHPIVAP20,NHBLACKAMINOTHVAP20,NHBLACKASIANNHPIVAP20,NHBLACKASIANOTHVAP20,NHBLACKNHPIOTHVAP20,NHAMINASIANNHPIVAP20,NHAMINASIANOTHVAP20,NHAMINNHPIOTHVAP20,NHASIANNHPIOTHVAP20,NHWHITEBLACKAMINASIANVAP20,NHWHITEBLACKAMINNHPIVAP20,NHWHITEBLACKAMINOTHVAP20,NHWHITEBLACKASIANNHPIVAP20,NHWHITEBLACKASIANOTHVAP20,NHWHITEBLACKNHPIOTHVAP20,NHWHITEAMINASIANNHPIVAP20,NHWHITEAMINASIANOTHVAP20,NHWHITEAMINNHPIOTHVAP20,NHWHITEASIANNHPIOTHVAP20,NHBLACKAMINASIANNHPIVAP20,NHBLACKAMINASIANOTHVAP20,NHBLACKAMINNHPIOTHVAP20,NHBLACKASIANNHPIOTHVAP20,NHAMINASIANNHPIOTHVAP20,NHWHITEBLACKAMINASIANNHPIVAP20,NHWHITEBLACKAMINASIANOTHVAP20,NHWHITEBLACKAMINNHPIOTHVAP20,NHWHITEBLACKASIANNHPIOTHVAP20,NHWHITEAMINASIANNHPIOTHVAP20,NHBLACKAMINASIANNHPIOTHVAP20,NHWHITEBLACKAMINASIANNHPIOTHVAP20,HWHITEVAP20,HBLACKVAP20,HAMINVAP20,HASIANVAP20,HNHPIVAP20,HOTHVAP20,HWHITEBLACKVAP20,HWHITEAMINVAP20,HWHITEASIANVAP20,HWHITENHPIVAP20,HWHITEOTHVAP20,HBLACKAMINVAP20,HBLACKASIANVAP20,HBLACKNHPIVAP20,HBLACKOTHVAP20,HAMINASIANVAP20,HAMINNHPIVAP20,HAMINOTHVAP20,HASIANNHPIVAP20,HASIANOTHVAP20,HNHPIOTHVAP20,HWHITEBLACKAMINVAP20,HWHITEBLACKASIANVAP20,HWHITEBLACKNHPIVAP20,HWHITEBLACKOTHVAP20,HWHITEAMINASIANVAP20,HWHITEAMINNHPIVAP20,HWHITEAMINOTHVAP20,HWHITEASIANNHPIVAP20,HWHITEASIANOTHVAP20,HWHITENHPIOTHVAP20,HBLACKAMINASIANVAP20,HBLACKAMINNHPIVAP20,HBLACKAMINOTHVAP20,HBLACKASIANNHPIVAP20,HBLACKASIANOTHVAP20,HBLACKNHPIOTHVAP20,HAMINASIANNHPIVAP20,HAMINASIANOTHVAP20,HAMINNHPIOTHVAP20,HASIANNHPIOTHVAP20,HWHITEBLACKAMINASIANVAP20,HWHITEBLACKAMINNHPIVAP20,HWHITEBLACKAMINOTHVAP20,HWHITEBLACKASIANNHPIVAP20,HWHITEBLACKASIANOTHVAP20,HWHITEBLACKNHPIOTHVAP20,HWHITEAMINASIANNHPIVAP20,HWHITEAMINASIANOTHVAP20,HWHITEAMINNHPIOTHVAP20,HWHITEASIANNHPIOTHVAP20,HBLACKAMINASIANNHPIVAP20,HBLACKAMINASIANOTHVAP20,HBLACKAMINNHPIOTHVAP20,HBLACKASIANNHPIOTHVAP20,HAMINASIANNHPIOTHVAP20,HWHITEBLACKAMINASIANNHPIVAP20,HWHITEBLACKAMINASIANOTHVAP20,HWHITEBLACKAMINNHPIOTHVAP20,HWHITEBLACKASIANNHPIOTHVAP20,HWHITEAMINASIANNHPIOTHVAP20,HBLACKAMINASIANNHPIOTHVAP20,HWHITEBLACKAMINASIANNHPIOTHVAP20,L_BVAP,L_HVAP,L_AAPIVAP,L_AMINVAP,L_WVAP
0,410670335003015,41067033500,3,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

### Just keep the columns we need:

In [17]:
census_vap_trimmed_df = census_vap_df[["GEOID20", "TRACT20", "VAP20", "L_HVAP", "L_AAPIVAP", "L_BVAP", "L_AMINVAP", "L_WVAP"]]

In [18]:
census_vap_trimmed_df

,GEOID20,TRACT20,VAP20,L_HVAP,L_AAPIVAP,L_BVAP,L_AMINVAP,L_WVAP
0,410670335003015,41067033500,3,0,0,1,0,2
1,410670335003016,41067033500,0,0,0,0,0,0
2,410710305012038,41071030501,16,2,1,0,3,10
3,410710305012039,41071030501,0,0,0,0,0,0
4,410710305012041,41071030501,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
130802,410510023033046,41051002303,0,0,0,0,0,0
130803,410510023033048,41051002303,0,0,0,0,0,0
130804,410510023033049,41051002303,301,35,50,12,1,203
130805,410510024011000,41051002401,40,0,0,6,1,33


### Rename the columns to be more standard:

In [19]:
census_vap_trimmed_df = census_vap_trimmed_df.rename(columns = {
    'L_HVAP': 'HVAP20',
    'L_AAPIVAP': 'AAPIVAP20',
    'L_BVAP': 'BVAP20',
    'L_AMINVAP': 'AMINVAP20',
    'L_WVAP': 'WVAP20'
})


In [20]:
census_vap_trimmed_df

,GEOID20,TRACT20,VAP20,HVAP20,AAPIVAP20,BVAP20,AMINVAP20,WVAP20
0,410670335003015,41067033500,3,0,0,1,0,2
1,410670335003016,41067033500,0,0,0,0,0,0
2,410710305012038,41071030501,16,2,1,0,3,10
3,410710305012039,41071030501,0,0,0,0,0,0
4,410710305012041,41071030501,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
130802,410510023033046,41051002303,0,0,0,0,0,0
130803,410510023033048,41051002303,0,0,0,0,0,0
130804,410510023033049,41051002303,301,35,50,12,1,203
130805,410510024011000,41051002401,40,0,0,6,1,33


### Now bring in ACS22 VAP/CVAP data:

In [21]:
tracts_acs5_22_df, tracts_acs5_22_moe_df = acs5(state, geometry = "tract", year = 2022)

In [22]:
tracts_acs5_22_df

,TOT_POP,TOT_VAP,WHITE_VAP,BLACK_VAP,AIAN_VAP,ASIAN_VAP,NHPI_VAP,OTH_VAP,2MORE_VAP,NHWHITE_VAP,HISP_VAP,TOT_CVAP,WHITE_CVAP,BLACK_CVAP,AIAN_CVAP,ASIAN_CVAP,NHPI_CVAP,OTH_CVAP,2MORE_CVAP,NHWHITE_CVAP,HISP_CVAP
GEO_ID,,,,,,,,,,,,,,,,,,,,,
41001950100,2659.0,2178.0,2061.0,19.0,16.0,4.0,18.0,33.0,27.0,2034.0,63.0,2155.0,2058.0,19.0,16.0,4.0,0.0,31.0,27.0,2031.0,61.0
41001950200,3305.0,2678.0,2534.0,30.0,8.0,8.0,4.0,13.0,81.0,2526.0,8.0,2678.0,2534.0,30.0,8.0,8.0,4.0,13.0,81.0,2526.0,8.0
41001950300,2874.0,2213.0,2036.0,44.0,68.0,4.0,13.0,3.0,45.0,2020.0,20.0,2213.0,2036.0,44.0,68.0,4.0,13.0,3.0,45.0,2020.0,20.0
41001950400,2910.0,2340.0,2253.0,21.0,0.0,20.0,0.0,27.0,19.0,2170.0,129.0,2296.0,2209.0,21.0,0.0,20.0,0.0,27.0,19.0,2170.0,85.0
41001950500,2661.0,2095.0,1861.0,12.0,56.0,0.0,0.0,53.0,113.0,1809.0,177.0,2040.0,1861.0,12.0,56.0,0.0,0.0,8.0,103.0,1809.0,122.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41071030704,4396.0,3314.0,2868.0,0.0,0.0,0.0,0.0,201.0,245.0,2821.0,317.0,3207.0,2868.0,0.0,0.0,0.0,0.0,102.0,237.0,2821.0,210.0
41071030801,5852.0,4499.0,3367.0,67.0,49.0,3.0,2.0,422.0,589.0,3138.0,950.0,4133.0,3272.0,67.0,49.0,1.0,2.0,176.0,566.0,3043.0,681.0
41071030802,4644.0,4019.0,2933.0,18.0,10.0,87.0,44.0,406.0,521.0,2711.0,785.0,3644.0,2853.0,18.0,10.0,52.0,44.0,168.0,499.0,2702.0,454.0


### I don't love this setting the GEOID equal to the index, so I'm going to undo it and rename the GEOID.

In [23]:
tracts_acs5_22_df = tracts_acs5_22_df.reset_index()
tracts_acs5_22_df = tracts_acs5_22_df.rename(columns = {'GEO_ID': 'TRACT20'})

In [24]:
list(tracts_acs5_22_df.columns)

['TRACT20',
 'TOT_POP',
 'TOT_VAP',
 'WHITE_VAP',
 'BLACK_VAP',
 'AIAN_VAP',
 'ASIAN_VAP',
 'NHPI_VAP',
 'OTH_VAP',
 '2MORE_VAP',
 'NHWHITE_VAP',
 'HISP_VAP',
 'TOT_CVAP',
 'WHITE_CVAP',
 'BLACK_CVAP',
 'AIAN_CVAP',
 'ASIAN_CVAP',
 'NHPI_CVAP',
 'OTH_CVAP',
 '2MORE_CVAP',
 'NHWHITE_CVAP',
 'HISP_CVAP']

### Rename some columns so they'll be more distinct from the decennial columns names.  (We don't need TOTPOP here, and I'm changing "2MORE" to "MULT" so it will play nicer with variable names.)

In [25]:
tracts_acs5_22_df = tracts_acs5_22_df.drop(columns = ['TOT_POP'])

tracts_acs5_22_df = tracts_acs5_22_df.rename(columns = {
    'TOT_VAP': 'VAP_ACS22',
    'WHITE_VAP': 'WHITEVAP_ACS22',
    'BLACK_VAP': 'BLACKVAP_ACS22',
    'AIAN_VAP': 'AMINVAP_ACS22',
    'ASIAN_VAP': 'ASIANVAP_ACS22',
    'NHPI_VAP': 'NHPIVAP_ACS22',
    'OTH_VAP': 'OTHVAP_ACS22',
    '2MORE_VAP': 'MULTVAP_ACS22',
    'NHWHITE_VAP': 'NHWHITEVAP_ACS22',
    'HISP_VAP': 'HVAP_ACS22',
    'TOT_CVAP': 'CVAP_ACS22',
    'WHITE_CVAP': 'WHITECVAP_ACS22',
    'BLACK_CVAP': 'BLACKCVAP_ACS22',
    'AIAN_CVAP': 'AMINCVAP_ACS22',
    'ASIAN_CVAP': 'ASIANCVAP_ACS22',
    'NHPI_CVAP': 'NHPICVAP_ACS22',
    'OTH_CVAP': 'OTHCVAP_ACS22',
    '2MORE_CVAP': 'MULTCVAP_ACS22',
    'NHWHITE_CVAP': 'NHWHITECVAP_ACS22',
    'HISP_CVAP': 'HCVAP_ACS22'
})

### Check that the racial categories - WITHOUT the HVAP and NHWHITEVAP columns! - sum to VAP/CVAP:

In [26]:
test1 = tracts_acs5_22_df[['WHITEVAP_ACS22', 'BLACKVAP_ACS22', 'AMINVAP_ACS22', 'ASIANVAP_ACS22', 'NHPIVAP_ACS22', 'OTHVAP_ACS22', 'MULTVAP_ACS22']].sum(axis=1) - tracts_acs5_22_df['VAP_ACS22']
[i for i in test1.index if test1[i] != 0]

[]

In [27]:
test2 = tracts_acs5_22_df[['WHITECVAP_ACS22', 'BLACKCVAP_ACS22', 'AMINCVAP_ACS22', 'ASIANCVAP_ACS22', 'NHPICVAP_ACS22', 'OTHCVAP_ACS22', 'MULTCVAP_ACS22']].sum(axis=1) - tracts_acs5_22_df['CVAP_ACS22']
[i for i in test2.index if test2[i] != 0]

[]

### So the ACS puts the Hispanic population BOTH in a racial category and in the Hispanic Category.  It also gives both WHITE and NHWHITE, from which we could infer HWHITE, but we have no way to tell how the rest of the Hispanic population is assigned to racial categories.

### Let's create categories for "AAPI" (ASIAN + NHPI) and "POC" (everybody except for NHWHITE).

In [28]:
tracts_acs5_22_df["AAPIVAP_ACS22"] = tracts_acs5_22_df["ASIANVAP_ACS22"] + tracts_acs5_22_df["NHPIVAP_ACS22"] 
tracts_acs5_22_df["AAPICVAP_ACS22"] = tracts_acs5_22_df["ASIANCVAP_ACS22"] + tracts_acs5_22_df["NHPICVAP_ACS22"] 

#tracts_acs5_22_df["POCVAP_ACS22"] = tracts_acs5_22_df["VAP_ACS22"] - tracts_acs5_22_df["NHWHITEVAP_ACS22"]
#tracts_acs5_22_df["POCCVAP_ACS22"] = tracts_acs5_22_df["CVAP_ACS22"] - tracts_acs5_22_df["NHWHITECVAP_ACS22"]


### Citizenship percentages, both at the tract level and statewide:

In [29]:
tracts_acs5_22_df["WHITE_cit_pct"] = tracts_acs5_22_df["WHITECVAP_ACS22"]/tracts_acs5_22_df["WHITEVAP_ACS22"]
tracts_acs5_22_df["BLACK_cit_pct"] = tracts_acs5_22_df["BLACKCVAP_ACS22"]/tracts_acs5_22_df["BLACKVAP_ACS22"]
tracts_acs5_22_df["AMIN_cit_pct"] = tracts_acs5_22_df["AMINCVAP_ACS22"]/tracts_acs5_22_df["AMINVAP_ACS22"]
tracts_acs5_22_df["ASIAN_cit_pct"] = tracts_acs5_22_df["ASIANCVAP_ACS22"]/tracts_acs5_22_df["ASIANVAP_ACS22"]
tracts_acs5_22_df["NHPI_cit_pct"] = tracts_acs5_22_df["NHPICVAP_ACS22"]/tracts_acs5_22_df["NHPIVAP_ACS22"]
tracts_acs5_22_df["AAPI_cit_pct"] = tracts_acs5_22_df["AAPICVAP_ACS22"]/tracts_acs5_22_df["AAPIVAP_ACS22"]
tracts_acs5_22_df["OTH_cit_pct"] = tracts_acs5_22_df["OTHCVAP_ACS22"]/tracts_acs5_22_df["OTHVAP_ACS22"]
tracts_acs5_22_df["MULT_cit_pct"] = tracts_acs5_22_df["MULTCVAP_ACS22"]/tracts_acs5_22_df["MULTVAP_ACS22"]
#tracts_acs5_22_df["POC_cit_pct"] = tracts_acs5_22_df["POCCVAP_ACS22"]/tracts_acs5_22_df["POCVAP_ACS22"]
tracts_acs5_22_df["HISP_cit_pct"] = tracts_acs5_22_df["HCVAP_ACS22"]/tracts_acs5_22_df["HVAP_ACS22"]
tracts_acs5_22_df["NHWHITE_cit_pct"] = tracts_acs5_22_df["NHWHITECVAP_ACS22"]/tracts_acs5_22_df["NHWHITEVAP_ACS22"]

WHITE_state_cit_pct = tracts_acs5_22_df["WHITECVAP_ACS22"].sum()/tracts_acs5_22_df["WHITEVAP_ACS22"].sum()
BLACK_state_cit_pct = tracts_acs5_22_df["BLACKCVAP_ACS22"].sum()/tracts_acs5_22_df["BLACKVAP_ACS22"].sum()
AMIN_state_cit_pct = tracts_acs5_22_df["AMINCVAP_ACS22"].sum()/tracts_acs5_22_df["AMINVAP_ACS22"].sum()
ASIAN_state_cit_pct = tracts_acs5_22_df["ASIANCVAP_ACS22"].sum()/tracts_acs5_22_df["ASIANVAP_ACS22"].sum()
NHPI_state_cit_pct = tracts_acs5_22_df["NHPICVAP_ACS22"].sum()/tracts_acs5_22_df["NHPIVAP_ACS22"].sum()
AAPI_state_cit_pct = tracts_acs5_22_df["AAPICVAP_ACS22"].sum()/tracts_acs5_22_df["AAPIVAP_ACS22"].sum()
OTH_state_cit_pct = tracts_acs5_22_df["OTHCVAP_ACS22"].sum()/tracts_acs5_22_df["OTHVAP_ACS22"].sum()
MULT_state_cit_pct = tracts_acs5_22_df["MULTCVAP_ACS22"].sum()/tracts_acs5_22_df["MULTVAP_ACS22"].sum()
#POC_state_cit_pct = tracts_acs5_22_df["POCCVAP_ACS22"].sum()/tracts_acs5_22_df["POCVAP_ACS22"].sum()
HISP_state_cit_pct = tracts_acs5_22_df["HCVAP_ACS22"].sum()/tracts_acs5_22_df["HVAP_ACS22"].sum()
NHWHITE_state_cit_pct = tracts_acs5_22_df["NHWHITECVAP_ACS22"].sum()/tracts_acs5_22_df["NHWHITEVAP_ACS22"].sum()


### Statewide total ACS VAP and citizenship percentages by racial category:

In [30]:
print('WHITE', tracts_acs5_22_df["WHITEVAP_ACS22"].sum(), WHITE_state_cit_pct)
print('BLACK', tracts_acs5_22_df["BLACKVAP_ACS22"].sum(), BLACK_state_cit_pct)
print('AMIN', tracts_acs5_22_df["AMINVAP_ACS22"].sum(), AMIN_state_cit_pct)
print('ASIAN', tracts_acs5_22_df["ASIANVAP_ACS22"].sum(), ASIAN_state_cit_pct)
print('NHPI', tracts_acs5_22_df["NHPIVAP_ACS22"].sum(), NHPI_state_cit_pct)
print('AAPI', tracts_acs5_22_df["AAPIVAP_ACS22"].sum(), AAPI_state_cit_pct)
print('OTH', tracts_acs5_22_df["OTHVAP_ACS22"].sum(), OTH_state_cit_pct)
print('MULT', tracts_acs5_22_df["MULTVAP_ACS22"].sum(), MULT_state_cit_pct)
#print('POC', tracts_acs5_22_df["POCVAP_ACS22"].sum(), POC_state_cit_pct)
print('HISP', tracts_acs5_22_df["HVAP_ACS22"].sum(), HISP_state_cit_pct)
print('NHWHITE', tracts_acs5_22_df["NHWHITEVAP_ACS22"].sum(), NHWHITE_state_cit_pct)


WHITE 2722817.0 0.974723971533893
BLACK 60766.0 0.9050949544152981
AMIN 36370.0 0.9180918339290625
ASIAN 155082.0 0.7234301853213139
NHPI 12754.0 0.756860592755214
AAPI 167836.0 0.7259705903381872
OTH 126888.0 0.6590378916840048
MULT 258719.0 0.8871478322040515
HISP 386871.0 0.7177948204957207
NHWHITE 2577634.0 0.9873438199527164


### Before we fill in NaN values, let's see, for each category, how many people of that category occur in decennial VAP for blocks that have no ACS VAP value in the corresponding tract.  (Ignore ASIAN and NHPI since they've been grouped together in AAPI, and MULT because we're not assigning any litigation category to it.)

In [31]:
acs_to_census_cats = {
    'BLACK': 'BVAP20',
    'HISP': 'HVAP20',
    'AMIN': 'AMINVAP20',
    'AAPI': 'AAPIVAP20',
    #'OTH': 'OTHVAP20',
    #'POC': 'POCVAP20',
    'NHWHITE': 'WVAP20'
}

In [32]:
for cat in acs_to_census_cats.keys():
    bad_tracts = list(tracts_acs5_22_df[tracts_acs5_22_df[cat+'_cit_pct'].isna()]['TRACT20'])
    bad_blocks_df = census_vap_trimmed_df[census_vap_trimmed_df['TRACT20'].isin(bad_tracts)]
    num_bad_tracts = len(bad_tracts)
    phantom_pop = bad_blocks_df[acs_to_census_cats[cat]].sum()
    total_pop = census_vap_trimmed_df[acs_to_census_cats[cat]].sum()
    print(cat, phantom_pop, phantom_pop/total_pop, num_bad_tracts)

BLACK 7590 0.09383228869191114 278
HISP 350 0.0008988325449287868 14
AMIN 19983 0.19933365918862034 271
AAPI 6409 0.029622610986572994 116
NHWHITE 32 1.2385027276087416e-05 9


### Oh wow, the ACS missed more than I would have thought!  

### Now fill in NaN values, replacing them by the statewide average for that column. Don't just fill with zeros since some blocks in those tracts have nonzero decennial census population.

In [33]:
tracts_acs5_22_df

,TRACT20,VAP_ACS22,WHITEVAP_ACS22,BLACKVAP_ACS22,AMINVAP_ACS22,ASIANVAP_ACS22,NHPIVAP_ACS22,OTHVAP_ACS22,MULTVAP_ACS22,NHWHITEVAP_ACS22,HVAP_ACS22,CVAP_ACS22,WHITECVAP_ACS22,BLACKCVAP_ACS22,AMINCVAP_ACS22,ASIANCVAP_ACS22,NHPICVAP_ACS22,OTHCVAP_ACS22,MULTCVAP_ACS22,NHWHITECVAP_ACS22,HCVAP_ACS22,AAPIVAP_ACS22,AAPICVAP_ACS22,WHITE_cit_pct,BLACK_cit_pct,AMIN_cit_pct,ASIAN_cit_pct,NHPI_cit_pct,AAPI_cit_pct,OTH_cit_pct,MULT_cit_pct,HISP_cit_pct,NHWHITE_cit_pct
0,41001950100,2178.0,2061.0,19.0,16.0,4.0,18.0,33.0,27.0,2034.0,63.0,2155.0,2058.0,19.0,16.0,4.0,0.0,31.0,27.0,2031.0,61.0,22.0,4.0,0.998544,1.0,1.0,1.000000,0.0,0.181818,0.939394,1.000000,0.968254,0.998525
1,41001950200,2678.0,2534.0,30.0,8.0,8.0,4.0,13.0,81.0,2526.0,8.0,2678.0,2534.0,30.0,8.0,8.0,4.0,13.0,81.0,2526.0,8.0,12.0,12.0,1.000000,1.0,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000
2,41001950300,2213.0,2036.0,44.0,68.0,4.0,13.0,3.0,45.0,2020.0,20.0,2213.0,2036.0,44.0,68.0,4.0,13.0,3.0,45.0,2020.0,20.0,17.0,17.0,1.000000,1.0,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000
3,41001950400,2340.0,2253.0,21.0,0.0,20.0,0.0,27.0,19.0,2170.0,129.0,2296.0,2209.0,21.0,0.0,20.0,0.0,27.0,19.0,2170.0,85.0,20.0,20.0,0.980470,1.0,NaN,1.000000,NaN,1.000000,1.000000,1.000000,0.658915,1.000000
4,41001950500,2095.0,1861.0,12.0,56.0,0.0,0.0,53.0,113.0,1809.0,177.0,2040.0,1861.0,12.0,56.0,0.0,0.0,8.0,103.0,1809.0,122.0,0.0,0.0,1.000000,1.0,1.0,NaN,NaN,NaN,0.150943,0.911504,0.689266,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,41071030704,3314.0,2868.0,0.0,0.0,0.0,0.0,201.0,245.0,2821.0,317.0,3207.0,2868.0,0.0,0.0,0.0,0.0,102.0,237.0,2821.0,210.0,0.0,0.0,1.000000,NaN,NaN,NaN,NaN,NaN,0.507463,0.967347,0.662461,1.000000
997,41071030801,4499.0,3367.0,67.0,49.0,3.0,2.0,422.0,589.0,3138.0,950.0,4133.0,3272.0,67.0,49.0,1.0,2.0,176.0,566.0,3043.0,681.0,5.0,3.0,0.971785,1.0,1.0,0.333333,1.0,0.600000,0.417062,0.960951,0.716842,0.969726
998,41071030802,4019.0,2933.0,18.0,10.0,87.0,44.0,406.0,521.0,2711.0,785.0,3644.0,2853.0,18.0,10.0,52.0,44.0,168.0,499.0,2702.0,454.0,131.0,96.0,0.972724,1.0,1.0,0.597701,1.0,0.732824,0.413793,0.957774,0.578344,0.996680
999,41071030900,3501.0,2558.0,0.0,17.0,0.0,0.0,308.0,618.0,2192.0,1271.0,2939.0,2512.0,0.0,17.0,0.0,0.0,151.0,259.0,2192.0,709.0,0.0,0.0,0.982017,NaN,1.0,NaN,NaN,NaN,0.490260,0.419094,0.557828,1.000000


In [34]:
fill_values = {"WHITE_cit_pct": WHITE_state_cit_pct,
               "BLACK_cit_pct": BLACK_state_cit_pct,
               "HISP_cit_pct": HISP_state_cit_pct,
               "AMIN_cit_pct": AMIN_state_cit_pct,
               "ASIAN_cit_pct": ASIAN_state_cit_pct,
               "NHPI_cit_pct": NHPI_state_cit_pct,
               "AAPI_cit_pct": AAPI_state_cit_pct,
               "OTH_cit_pct": OTH_state_cit_pct,
               "MULT_cit_pct": MULT_state_cit_pct,
               #"POC_cit_pct": POC_state_cit_pct,
               "NHWHITE_cit_pct": NHWHITE_state_cit_pct}
          
    
tracts_acs5_22_df = tracts_acs5_22_df.fillna(value = fill_values)

In [35]:
tracts_acs5_22_df

,TRACT20,VAP_ACS22,WHITEVAP_ACS22,BLACKVAP_ACS22,AMINVAP_ACS22,ASIANVAP_ACS22,NHPIVAP_ACS22,OTHVAP_ACS22,MULTVAP_ACS22,NHWHITEVAP_ACS22,HVAP_ACS22,CVAP_ACS22,WHITECVAP_ACS22,BLACKCVAP_ACS22,AMINCVAP_ACS22,ASIANCVAP_ACS22,NHPICVAP_ACS22,OTHCVAP_ACS22,MULTCVAP_ACS22,NHWHITECVAP_ACS22,HCVAP_ACS22,AAPIVAP_ACS22,AAPICVAP_ACS22,WHITE_cit_pct,BLACK_cit_pct,AMIN_cit_pct,ASIAN_cit_pct,NHPI_cit_pct,AAPI_cit_pct,OTH_cit_pct,MULT_cit_pct,HISP_cit_pct,NHWHITE_cit_pct
0,41001950100,2178.0,2061.0,19.0,16.0,4.0,18.0,33.0,27.0,2034.0,63.0,2155.0,2058.0,19.0,16.0,4.0,0.0,31.0,27.0,2031.0,61.0,22.0,4.0,0.998544,1.000000,1.000000,1.000000,0.000000,0.181818,0.939394,1.000000,0.968254,0.998525
1,41001950200,2678.0,2534.0,30.0,8.0,8.0,4.0,13.0,81.0,2526.0,8.0,2678.0,2534.0,30.0,8.0,8.0,4.0,13.0,81.0,2526.0,8.0,12.0,12.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,41001950300,2213.0,2036.0,44.0,68.0,4.0,13.0,3.0,45.0,2020.0,20.0,2213.0,2036.0,44.0,68.0,4.0,13.0,3.0,45.0,2020.0,20.0,17.0,17.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,41001950400,2340.0,2253.0,21.0,0.0,20.0,0.0,27.0,19.0,2170.0,129.0,2296.0,2209.0,21.0,0.0,20.0,0.0,27.0,19.0,2170.0,85.0,20.0,20.0,0.980470,1.000000,0.918092,1.000000,0.756861,1.000000,1.000000,1.000000,0.658915,1.000000
4,41001950500,2095.0,1861.0,12.0,56.0,0.0,0.0,53.0,113.0,1809.0,177.0,2040.0,1861.0,12.0,56.0,0.0,0.0,8.0,103.0,1809.0,122.0,0.0,0.0,1.000000,1.000000,1.000000,0.723430,0.756861,0.725971,0.150943,0.911504,0.689266,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,41071030704,3314.0,2868.0,0.0,0.0,0.0,0.0,201.0,245.0,2821.0,317.0,3207.0,2868.0,0.0,0.0,0.0,0.0,102.0,237.0,2821.0,210.0,0.0,0.0,1.000000,0.905095,0.918092,0.723430,0.756861,0.725971,0.507463,0.967347,0.662461,1.000000
997,41071030801,4499.0,3367.0,67.0,49.0,3.0,2.0,422.0,589.0,3138.0,950.0,4133.0,3272.0,67.0,49.0,1.0,2.0,176.0,566.0,3043.0,681.0,5.0,3.0,0.971785,1.000000,1.000000,0.333333,1.000000,0.600000,0.417062,0.960951,0.716842,0.969726
998,41071030802,4019.0,2933.0,18.0,10.0,87.0,44.0,406.0,521.0,2711.0,785.0,3644.0,2853.0,18.0,10.0,52.0,44.0,168.0,499.0,2702.0,454.0,131.0,96.0,0.972724,1.000000,1.000000,0.597701,1.000000,0.732824,0.413793,0.957774,0.578344,0.996680
999,41071030900,3501.0,2558.0,0.0,17.0,0.0,0.0,308.0,618.0,2192.0,1271.0,2939.0,2512.0,0.0,17.0,0.0,0.0,151.0,259.0,2192.0,709.0,0.0,0.0,0.982017,0.905095,1.000000,0.723430,0.756861,0.725971,0.490260,0.419094,0.557828,1.000000


### Now add columns for DVAP - note that total DVAP will be computed as the sum of categorical DVAP, and we're going to do this two ways, once with broken-down racial categories and once with just POC, so we'll need TWO total DVAP columns:

In [36]:
census_vap_trimmed_df

,GEOID20,TRACT20,VAP20,HVAP20,AAPIVAP20,BVAP20,AMINVAP20,WVAP20
0,410670335003015,41067033500,3,0,0,1,0,2
1,410670335003016,41067033500,0,0,0,0,0,0
2,410710305012038,41071030501,16,2,1,0,3,10
3,410710305012039,41071030501,0,0,0,0,0,0
4,410710305012041,41071030501,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
130802,410510023033046,41051002303,0,0,0,0,0,0
130803,410510023033048,41051002303,0,0,0,0,0,0
130804,410510023033049,41051002303,301,35,50,12,1,203
130805,410510024011000,41051002401,40,0,0,6,1,33


In [37]:
census_vap_trimmed_df.insert(3, 'DVAP22', None)
census_vap_trimmed_df.insert(9, 'HDVAP22', None)
census_vap_trimmed_df.insert(10, 'AAPIDVAP22', None)
census_vap_trimmed_df.insert(11, 'BDVAP22', None)
census_vap_trimmed_df.insert(12, 'AMINDVAP22', None)
census_vap_trimmed_df.insert(13, 'WDVAP22', None)

In [38]:
census_vap_trimmed_df

,GEOID20,TRACT20,VAP20,DVAP22,HVAP20,AAPIVAP20,BVAP20,AMINVAP20,WVAP20,HDVAP22,AAPIDVAP22,BDVAP22,AMINDVAP22,WDVAP22
0,410670335003015,41067033500,3,None,0,0,1,0,2,None,None,None,None,None
1,410670335003016,41067033500,0,None,0,0,0,0,0,None,None,None,None,None
2,410710305012038,41071030501,16,None,2,1,0,3,10,None,None,None,None,None
3,410710305012039,41071030501,0,None,0,0,0,0,0,None,None,None,None,None
4,410710305012041,41071030501,0,None,0,0,0,0,0,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130802,410510023033046,41051002303,0,None,0,0,0,0,0,None,None,None,None,None
130803,410510023033048,41051002303,0,None,0,0,0,0,0,None,None,None,None,None
130804,410510023033049,41051002303,301,None,35,50,12,1,203,None,None,None,None,None
130805,410510024011000,41051002401,40,None,0,0,6,1,33,None,None,None,None,None


### It will be useful to index the tracts by their TRACT20 ID.

In [39]:
tracts_df = tracts_acs5_22_df.set_index('TRACT20')

In [40]:
tracts_df

,VAP_ACS22,WHITEVAP_ACS22,BLACKVAP_ACS22,AMINVAP_ACS22,ASIANVAP_ACS22,NHPIVAP_ACS22,OTHVAP_ACS22,MULTVAP_ACS22,NHWHITEVAP_ACS22,HVAP_ACS22,CVAP_ACS22,WHITECVAP_ACS22,BLACKCVAP_ACS22,AMINCVAP_ACS22,ASIANCVAP_ACS22,NHPICVAP_ACS22,OTHCVAP_ACS22,MULTCVAP_ACS22,NHWHITECVAP_ACS22,HCVAP_ACS22,AAPIVAP_ACS22,AAPICVAP_ACS22,WHITE_cit_pct,BLACK_cit_pct,AMIN_cit_pct,ASIAN_cit_pct,NHPI_cit_pct,AAPI_cit_pct,OTH_cit_pct,MULT_cit_pct,HISP_cit_pct,NHWHITE_cit_pct
TRACT20,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41001950100,2178.0,2061.0,19.0,16.0,4.0,18.0,33.0,27.0,2034.0,63.0,2155.0,2058.0,19.0,16.0,4.0,0.0,31.0,27.0,2031.0,61.0,22.0,4.0,0.998544,1.000000,1.000000,1.000000,0.000000,0.181818,0.939394,1.000000,0.968254,0.998525
41001950200,2678.0,2534.0,30.0,8.0,8.0,4.0,13.0,81.0,2526.0,8.0,2678.0,2534.0,30.0,8.0,8.0,4.0,13.0,81.0,2526.0,8.0,12.0,12.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
41001950300,2213.0,2036.0,44.0,68.0,4.0,13.0,3.0,45.0,2020.0,20.0,2213.0,2036.0,44.0,68.0,4.0,13.0,3.0,45.0,2020.0,20.0,17.0,17.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
41001950400,2340.0,2253.0,21.0,0.0,20.0,0.0,27.0,19.0,2170.0,129.0,2296.0,2209.0,21.0,0.0,20.0,0.0,27.0,19.0,2170.0,85.0,20.0,20.0,0.980470,1.000000,0.918092,1.000000,0.756861,1.000000,1.000000,1.000000,0.658915,1.000000
41001950500,2095.0,1861.0,12.0,56.0,0.0,0.0,53.0,113.0,1809.0,177.0,2040.0,1861.0,12.0,56.0,0.0,0.0,8.0,103.0,1809.0,122.0,0.0,0.0,1.000000,1.000000,1.000000,0.723430,0.756861,0.725971,0.150943,0.911504,0.689266,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41071030704,3314.0,2868.0,0.0,0.0,0.0,0.0,201.0,245.0,2821.0,317.0,3207.0,2868.0,0.0,0.0,0.0,0.0,102.0,237.0,2821.0,210.0,0.0,0.0,1.000000,0.905095,0.918092,0.723430,0.756861,0.725971,0.507463,0.967347,0.662461,1.000000
41071030801,4499.0,3367.0,67.0,49.0,3.0,2.0,422.0,589.0,3138.0,950.0,4133.0,3272.0,67.0,49.0,1.0,2.0,176.0,566.0,3043.0,681.0,5.0,3.0,0.971785,1.000000,1.000000,0.333333,1.000000,0.600000,0.417062,0.960951,0.716842,0.969726
41071030802,4019.0,2933.0,18.0,10.0,87.0,44.0,406.0,521.0,2711.0,785.0,3644.0,2853.0,18.0,10.0,52.0,44.0,168.0,499.0,2702.0,454.0,131.0,96.0,0.972724,1.000000,1.000000,0.597701,1.000000,0.732824,0.413793,0.957774,0.578344,0.996680


### ACS racial/ethnic groupings and citizenship percentage columns:

In [41]:
pop_cat_cols = ['HVAP20', 'AAPIVAP20', 'BVAP20', 'AMINVAP20', 'WVAP20']
dvap_cols = [col[:-5]+"DVAP22" for col in pop_cat_cols]

In [42]:
acs_cit_pct = {
    'HVAP20': "HISP_cit_pct",
    'AAPIVAP20': "AAPI_cit_pct",
    'BVAP20': "BLACK_cit_pct",
    'AMINVAP20': "AMIN_cit_pct",
    'WVAP20': "NHWHITE_cit_pct"
}

In [43]:
for i in tqdm(census_vap_trimmed_df.index):
    for col in pop_cat_cols:
        dcol = col[:-5]+"DVAP22"
        tractID = census_vap_trimmed_df.loc[i, "TRACT20"]
        cit_pct = tracts_df.loc[tractID, acs_cit_pct[col]]
        census_vap_trimmed_df.loc[i, dcol] = census_vap_trimmed_df.loc[i, col]*cit_pct


100%|█████████████████████████████████| 130807/130807 [00:34<00:00, 3785.65it/s]


In [44]:
census_vap_trimmed_df['DVAP22'] = census_vap_trimmed_df[['HDVAP22', 'AAPIDVAP22', 'BDVAP22', 'AMINDVAP22', 'WDVAP22']].sum(axis=1)


In [45]:
census_vap_trimmed_df

,GEOID20,TRACT20,VAP20,DVAP22,HVAP20,AAPIVAP20,BVAP20,AMINVAP20,WVAP20,HDVAP22,AAPIDVAP22,BDVAP22,AMINDVAP22,WDVAP22
0,410670335003015,41067033500,3,2.992029,0,0,1,0,2,0.0,0.0,1.0,0.0,1.992029
1,410670335003016,41067033500,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
2,410710305012038,41071030501,16,14.966499,2,1,0,3,10,1.565637,1.0,0.0,2.524887,9.875975
3,410710305012039,41071030501,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
4,410710305012041,41071030501,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130802,410510023033046,41051002303,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
130803,410510023033048,41051002303,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
130804,410510023033049,41051002303,301,279.613296,35,50,12,1,203,29.85623,40.039841,9.836066,1.0,198.881159
130805,410510024011000,41051002401,40,39.199611,0,0,6,1,33,0.0,0.0,6.0,0.918092,32.281519


### Total DVAP, and as a percentage of VAP?

In [46]:
print(census_vap_trimmed_df['DVAP22'].sum())
print(census_vap_trimmed_df['VAP20'].sum())
print(census_vap_trimmed_df['DVAP22'].sum()/census_vap_trimmed_df['VAP20'].sum())

3160726.465702268
3370652
0.9377196060887532


### Compare to ACS:

In [47]:
print(tracts_df['VAP_ACS22'].sum())
print(tracts_df['CVAP_ACS22'].sum())
print(tracts_df['CVAP_ACS22'].sum()/tracts_df['VAP_ACS22'].sum())


3373396.0
3177375.0
0.9418920873801949


### Check data type of DVAP columns:

In [48]:
census_vap_trimmed_df['VAP20']

0           3
1           0
2          16
3           0
4           0
         ... 
130802      0
130803      0
130804    301
130805     40
130806     34
Name: VAP20, Length: 130807, dtype: int64

### Okay, these all need to be converted to floats!

In [49]:
dvap_cols

['HDVAP22', 'AAPIDVAP22', 'BDVAP22', 'AMINDVAP22', 'WDVAP22']

In [50]:
for col in ["DVAP22"] + dvap_cols:
    census_vap_trimmed_df[col] = census_vap_trimmed_df[col].astype(float)

### Check a random tract:   

In [51]:
tracts_df

,VAP_ACS22,WHITEVAP_ACS22,BLACKVAP_ACS22,AMINVAP_ACS22,ASIANVAP_ACS22,NHPIVAP_ACS22,OTHVAP_ACS22,MULTVAP_ACS22,NHWHITEVAP_ACS22,HVAP_ACS22,CVAP_ACS22,WHITECVAP_ACS22,BLACKCVAP_ACS22,AMINCVAP_ACS22,ASIANCVAP_ACS22,NHPICVAP_ACS22,OTHCVAP_ACS22,MULTCVAP_ACS22,NHWHITECVAP_ACS22,HCVAP_ACS22,AAPIVAP_ACS22,AAPICVAP_ACS22,WHITE_cit_pct,BLACK_cit_pct,AMIN_cit_pct,ASIAN_cit_pct,NHPI_cit_pct,AAPI_cit_pct,OTH_cit_pct,MULT_cit_pct,HISP_cit_pct,NHWHITE_cit_pct
TRACT20,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41001950100,2178.0,2061.0,19.0,16.0,4.0,18.0,33.0,27.0,2034.0,63.0,2155.0,2058.0,19.0,16.0,4.0,0.0,31.0,27.0,2031.0,61.0,22.0,4.0,0.998544,1.000000,1.000000,1.000000,0.000000,0.181818,0.939394,1.000000,0.968254,0.998525
41001950200,2678.0,2534.0,30.0,8.0,8.0,4.0,13.0,81.0,2526.0,8.0,2678.0,2534.0,30.0,8.0,8.0,4.0,13.0,81.0,2526.0,8.0,12.0,12.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
41001950300,2213.0,2036.0,44.0,68.0,4.0,13.0,3.0,45.0,2020.0,20.0,2213.0,2036.0,44.0,68.0,4.0,13.0,3.0,45.0,2020.0,20.0,17.0,17.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
41001950400,2340.0,2253.0,21.0,0.0,20.0,0.0,27.0,19.0,2170.0,129.0,2296.0,2209.0,21.0,0.0,20.0,0.0,27.0,19.0,2170.0,85.0,20.0,20.0,0.980470,1.000000,0.918092,1.000000,0.756861,1.000000,1.000000,1.000000,0.658915,1.000000
41001950500,2095.0,1861.0,12.0,56.0,0.0,0.0,53.0,113.0,1809.0,177.0,2040.0,1861.0,12.0,56.0,0.0,0.0,8.0,103.0,1809.0,122.0,0.0,0.0,1.000000,1.000000,1.000000,0.723430,0.756861,0.725971,0.150943,0.911504,0.689266,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41071030704,3314.0,2868.0,0.0,0.0,0.0,0.0,201.0,245.0,2821.0,317.0,3207.0,2868.0,0.0,0.0,0.0,0.0,102.0,237.0,2821.0,210.0,0.0,0.0,1.000000,0.905095,0.918092,0.723430,0.756861,0.725971,0.507463,0.967347,0.662461,1.000000
41071030801,4499.0,3367.0,67.0,49.0,3.0,2.0,422.0,589.0,3138.0,950.0,4133.0,3272.0,67.0,49.0,1.0,2.0,176.0,566.0,3043.0,681.0,5.0,3.0,0.971785,1.000000,1.000000,0.333333,1.000000,0.600000,0.417062,0.960951,0.716842,0.969726
41071030802,4019.0,2933.0,18.0,10.0,87.0,44.0,406.0,521.0,2711.0,785.0,3644.0,2853.0,18.0,10.0,52.0,44.0,168.0,499.0,2702.0,454.0,131.0,96.0,0.972724,1.000000,1.000000,0.597701,1.000000,0.732824,0.413793,0.957774,0.578344,0.996680


In [52]:
test_blocks_df = census_vap_trimmed_df[census_vap_trimmed_df["TRACT20"] == "41071030801"]

In [53]:
test_blocks_df

,GEOID20,TRACT20,VAP20,DVAP22,HVAP20,AAPIVAP20,BVAP20,AMINVAP20,WVAP20,HDVAP22,AAPIDVAP22,BDVAP22,AMINDVAP22,WDVAP22
8610,410710308011000,41071030801,0,0.000000,0,0,0,0,0,0.000000,0.0,0.0,0.0,0.000000
8611,410710308011001,41071030801,219,203.411096,36,1,6,11,165,25.806316,0.6,6.0,11.0,160.004780
8612,410710308011003,41071030801,25,21.461426,11,0,0,0,14,7.885263,0.0,0.0,0.0,13.576163
8613,410710308011004,41071030801,21,17.231996,8,3,0,0,10,5.734737,1.8,0.0,0.0,9.697259
9746,410710308011006,41071030801,426,406.551725,22,3,0,4,397,15.770526,1.8,0.0,4.0,384.981198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47040,410710308011015,41071030801,3,2.969726,0,0,0,2,1,0.000000,0.0,0.0,2.0,0.969726
47041,410710308012003,41071030801,6,5.226020,1,1,0,1,3,0.716842,0.6,0.0,1.0,2.909178
47042,410710308012006,41071030801,1,0.716842,1,0,0,0,0,0.716842,0.0,0.0,0.0,0.000000
47043,410710308012009,41071030801,9,7.210230,6,0,0,0,3,4.301053,0.0,0.0,0.0,2.909178


In [54]:
print(tracts_df.loc["41071030801", "HISP_cit_pct"])
test_list = []
for i in test_blocks_df.index:
    if test_blocks_df.loc[i, "HVAP20"] > 0:
        test_list.append(test_blocks_df.loc[i, "HDVAP22"]/test_blocks_df.loc[i, "HVAP20"])

print(min(test_list), max(test_list))

0.716842105263158
0.716842105263158 0.716842105263158


In [55]:
print(tracts_df.loc["41071030801", "BLACK_cit_pct"])
test_list = []
for i in test_blocks_df.index:
    if test_blocks_df.loc[i, "BVAP20"] > 0:
        test_list.append(test_blocks_df.loc[i, "BDVAP22"]/test_blocks_df.loc[i, "BVAP20"])

print(min(test_list), max(test_list))

1.0
1.0 1.0


In [56]:
print(tracts_df.loc["41071030801", "AAPI_cit_pct"])
test_list = []
for i in test_blocks_df.index:
    if test_blocks_df.loc[i, "AAPIVAP20"] > 0:
        test_list.append(test_blocks_df.loc[i, "AAPIDVAP22"]/test_blocks_df.loc[i, "AAPIVAP20"])

print(min(test_list), max(test_list))

0.6
0.6 0.6


### Looks good to me!  Select/rearrange columns and save to csv.

In [57]:
census_vap_trimmed_df.columns

Index(['GEOID20', 'TRACT20', 'VAP20', 'DVAP22', 'HVAP20', 'AAPIVAP20',
       'BVAP20', 'AMINVAP20', 'WVAP20', 'HDVAP22', 'AAPIDVAP22', 'BDVAP22',
       'AMINDVAP22', 'WDVAP22'],
      dtype='object')

In [58]:
census_vap_trimmed_df = census_vap_trimmed_df[['GEOID20', 'TRACT20', 'VAP20', 'HVAP20', 'AAPIVAP20', 'BVAP20', 
                                               'AMINVAP20', 'WVAP20', 
                                               'DVAP22', 'HDVAP22', 'AAPIDVAP22', 'BDVAP22', 'AMINDVAP22', 
                                               'WDVAP22']]

In [59]:
census_vap_trimmed_df.to_csv("./Data/Shapefiles/OR_blocks/OR_blocks_DVAP.csv")

### Now add it to the Oregon block shapefile and save that too.

In [60]:
OR_blocks_df = gpd.read_file("./Data/Shapefiles/or_pl2020_b/or_pl2020_p1_b.shp")

In [61]:
OR_blocks_df

,GEOID20,SUMLEV,LOGRECNO,GEOID,COUNTY,P0010001,P0010002,P0010003,P0010004,P0010005,P0010006,P0010007,P0010008,P0010009,P0010010,P0010011,P0010012,P0010013,P0010014,P0010015,P0010016,P0010017,P0010018,P0010019,P0010020,P0010021,P0010022,P0010023,P0010024,P0010025,P0010026,P0010027,P0010028,P0010029,P0010030,P0010031,P0010032,P0010033,P0010034,P0010035,P0010036,P0010037,P0010038,P0010039,P0010040,P0010041,P0010042,P0010043,P0010044,P0010045,P0010046,P0010047,P0010048,P0010049,P0010050,P0010051,P0010052,P0010053,P0010054,P0010055,P0010056,P0010057,P0010058,P0010059,P0010060,P0010061,P0010062,P0010063,P0010064,P0010065,P0010066,P0010067,P0010068,P0010069,P0010070,P0010071,geometry
0,410159502021019,750,0046915,7500000US410159502021019,015,28,22,18,0,0,0,0,4,6,4,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((-124.43480 42.43459, -124.43458 42.4..."
1,410159501001119,750,0046036,7500000US410159501001119,015,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((-123.78119 42.69502, -123.78105 42.6..."
2,410159503014130,750,0047285,7500000US410159503014130,015,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((-124.21478 42.32879, -124.21467 42.3..."
3,410159504012139,750,0047754,7500000US410159504012139,015,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((-124.19303 41.99814, -124.19284 41.9..."
4,410159501001102,750,0046019,7500000US410159501001102,015,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((-123.78283 42.74463, -123.78262 42.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130802,410079509002019,750,0037795,7500000US410079509002019,007,2,0,0,0,0,0,0,0,2,2,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((-123.91419 46.00042, -123.91418 46.0..."
130803,410079511001096,750,0038044,7500000US410079511001096,007,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((-123.91179 46.00169, -123.91174 46.0..."
130804,410579601021015,750,0132421,7500000US410579601021015,057,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((-123.36535 45.76447, -123.36529 45.7..."
130805,410579601021079,750,0132485,7500000US410579601021079,057,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((-123.84320 45.75029, -123.84304 45.7..."


In [62]:
OR_blocks_trimmed_df = OR_blocks_df[['GEOID20', 'COUNTY', 'P0010001', 'geometry']]

In [63]:
OR_blocks_trimmed_df = OR_blocks_trimmed_df.rename(columns = {'P0010001': 'TOTPOP20'})

In [64]:
OR_blocks_trimmed_df.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands

### Convert to a non-geographic CRS:

In [65]:
OR_blocks_trimmed_df = OR_blocks_trimmed_df.to_crs(OR_blocks_df.estimate_utm_crs())

In [66]:
OR_blocks_trimmed_df.crs

<Projected CRS: EPSG:32610>
Name: WGS 84 / UTM zone 10N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 126°W and 120°W, northern hemisphere between equator and 84°N, onshore and offshore. Canada - British Columbia (BC); Northwest Territories (NWT); Nunavut; Yukon. United States (USA) - Alaska (AK).
- bounds: (-126.0, 0.0, -120.0, 84.0)
Coordinate Operation:
- name: UTM zone 10N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### Merge with VAP/DVAP data:

In [67]:
OR_blocks_trimmed_df['GEOID20']

0         410159502021019
1         410159501001119
2         410159503014130
3         410159504012139
4         410159501001102
               ...       
130802    410079509002019
130803    410079511001096
130804    410579601021015
130805    410579601021079
130806    410510082012014
Name: GEOID20, Length: 130807, dtype: object

In [68]:
census_vap_trimmed_df['GEOID20']

0         410670335003015
1         410670335003016
2         410710305012038
3         410710305012039
4         410710305012041
               ...       
130802    410510023033046
130803    410510023033048
130804    410510023033049
130805    410510024011000
130806    410510024011001
Name: GEOID20, Length: 130807, dtype: object

In [69]:
OR_blocks_trimmed_df = OR_blocks_trimmed_df.merge(census_vap_trimmed_df, on = 'GEOID20')

### Reorder the columns:

In [70]:
OR_blocks_trimmed_df.columns

Index(['GEOID20', 'COUNTY', 'TOTPOP20', 'geometry', 'TRACT20', 'VAP20',
       'HVAP20', 'AAPIVAP20', 'BVAP20', 'AMINVAP20', 'WVAP20', 'DVAP22',
       'HDVAP22', 'AAPIDVAP22', 'BDVAP22', 'AMINDVAP22', 'WDVAP22'],
      dtype='object')

In [71]:
OR_blocks_trimmed_df = OR_blocks_trimmed_df[['GEOID20', 'TRACT20', 'COUNTY', 'TOTPOP20', 'VAP20', 
                                             'HVAP20', 'AAPIVAP20', 'BVAP20', 'AMINVAP20', 'WVAP20', 
                                             'DVAP22', 'HDVAP22', 'AAPIDVAP22', 'BDVAP22', 'AMINDVAP22', 
                                             'WDVAP22', 'geometry']]

### Save it!

In [72]:
OR_blocks_trimmed_df.to_file("./Data/Shapefiles/OR_blocks/OR_blocks_with_VAP_and_DVAP.shp")

### Compute some racial percentages of VAP/CVAP for both decennial and ACS data to see how they compare:

### Black:

In [73]:
print(census_vap_trimmed_df['BVAP20'].sum()/census_vap_trimmed_df['VAP20'].sum())
print(tracts_df['BLACKVAP_ACS22'].sum()/tracts_df['VAP_ACS22'].sum())
print(tracts_df['BLACKCVAP_ACS22'].sum()/tracts_df['CVAP_ACS22'].sum())

0.023998027681291334
0.018013301729177363
0.01730957158031394


### Hispanic:

In [74]:
print(census_vap_trimmed_df['HVAP20'].sum()/census_vap_trimmed_df['VAP20'].sum())
print(tracts_df['HVAP_ACS22'].sum()/tracts_df['VAP_ACS22'].sum())
print(tracts_df['HCVAP_ACS22'].sum()/tracts_df['CVAP_ACS22'].sum())

0.11552483021089095
0.11468294857763511
0.08739730123136237


### AAPI:

In [75]:
print(census_vap_trimmed_df['AAPIVAP20'].sum()/census_vap_trimmed_df['VAP20'].sum())
print(tracts_df['AAPIVAP_ACS22'].sum()/tracts_df['VAP_ACS22'].sum())
print(tracts_df['AAPICVAP_ACS22'].sum()/tracts_df['CVAP_ACS22'].sum())

0.06418787819092567
0.049752830678639566
0.03834737794563122


### AMIN:

In [76]:
print(census_vap_trimmed_df['AMINVAP20'].sum()/census_vap_trimmed_df['VAP20'].sum())
print(tracts_df['AMINVAP_ACS22'].sum()/tracts_df['VAP_ACS22'].sum())
print(tracts_df['AMINCVAP_ACS22'].sum()/tracts_df['CVAP_ACS22'].sum())

0.029741723559714855
0.010781420266105728
0.010508989338683663


### NH White + Other:

In [77]:
print(census_vap_trimmed_df['WVAP20'].sum()/census_vap_trimmed_df['VAP20'].sum())
print(tracts_df['NHWHITEVAP_ACS22'].sum()/tracts_df['VAP_ACS22'].sum())
print(tracts_df['NHWHITECVAP_ACS22'].sum()/tracts_df['CVAP_ACS22'].sum())

0.7665475403571772
0.7641065561232657
0.8009791101144813


### Curious: in the decennial data, how does each grouped racial category split between Hispanic/non-Hispanic?  And how many people are in each pairwise overlap?

In [78]:
vap_df = census_vap_df[['VAP20', 'HVAP20'] + all_cols]
vap_df['NHVAP20'] = vap_df['VAP20'] - vap_df['HVAP20']

In [79]:
vap_df

,VAP20,HVAP20,HWHITEVAP20,HBLACKVAP20,HAMINVAP20,HASIANVAP20,HNHPIVAP20,HOTHVAP20,HWHITEBLACKVAP20,HWHITEAMINVAP20,HWHITEASIANVAP20,HWHITENHPIVAP20,HWHITEOTHVAP20,HBLACKAMINVAP20,HBLACKASIANVAP20,HBLACKNHPIVAP20,HBLACKOTHVAP20,HAMINASIANVAP20,HAMINNHPIVAP20,HAMINOTHVAP20,HASIANNHPIVAP20,HASIANOTHVAP20,HNHPIOTHVAP20,HWHITEBLACKAMINVAP20,HWHITEBLACKASIANVAP20,HWHITEBLACKNHPIVAP20,HWHITEBLACKOTHVAP20,HWHITEAMINASIANVAP20,HWHITEAMINNHPIVAP20,HWHITEAMINOTHVAP20,HWHITEASIANNHPIVAP20,HWHITEASIANOTHVAP20,HWHITENHPIOTHVAP20,HBLACKAMINASIANVAP20,HBLACKAMINNHPIVAP20,HBLACKAMINOTHVAP20,HBLACKASIANNHPIVAP20,HBLACKASIANOTHVAP20,HBLACKNHPIOTHVAP20,HAMINASIANNHPIVAP20,HAMINASIANOTHVAP20,HAMINNHPIOTHVAP20,HASIANNHPIOTHVAP20,HWHITEBLACKAMINASIANVAP20,HWHITEBLACKAMINNHPIVAP20,HWHITEBLACKAMINOTHVAP20,HWHITEBLACKASIANNHPIVAP20,HWHITEBLACKASIANOTHVAP20,HWHITEBLACKNHPIOTHVAP20,HWHITEAMINASIANNHPIVAP20,HWHITEAMINASIANOTHVAP20,HWHITEAMINNHPIOTHVAP20,HWHITEASIANNHPIOTHVAP20,HBLACKAMINASIANNHPIVAP20,HBLACKAMINASIANOTHVAP20,HBLACKAMINNHPIOTHVAP20,HBLACKASIANNHPIOTHVAP20,HAMINASIANNHPIOTHVAP20,HWHITEBLACKAMINASIANNHPIVAP20,HWHITEBLACKAMINASIANOTHVAP20,HWHITEBLACKAMINNHPIOTHVAP20,HWHITEBLACKASIANNHPIOTHVAP20,HWHITEAMINASIANNHPIOTHVAP20,HBLACKAMINASIANNHPIOTHVAP20,HWHITEBLACKAMINASIANNHPIOTHVAP20,NHWHITEVAP20,NHBLACKVAP20,NHAMINVAP20,NHASIANVAP20,NHNHPIVAP20,NHOTHVAP20,NHWHITEBLACKVAP20,NHWHITEAMINVAP20,NHWHITEASIANVAP20,NHWHITENHPIVAP20,NHWHITEOTHVAP20,NHBLACKAMINVAP20,NHBLACKASIANVAP20,NHBLACKNHPIVAP20,NHBLACKOTHVAP20,NHAMINASIANVAP20,NHAMINNHPIVAP20,NHAMINOTHVAP20,NHASIANNHPIVAP20,NHASIANOTHVAP20,NHNHPIOTHVAP20,NHWHITEBLACKAMINVAP20,NHWHITEBLACKASIANVAP20,NHWHITEBLACKNHPIVAP20,NHWHITEBLACKOTHVAP20,NHWHITEAMINASIANVAP20,NHWHITEAMINNHPIVAP20,NHWHITEAMINOTHVAP20,NHWHITEASIANNHPIVAP20,NHWHITEASIANOTHVAP20,NHWHITENHPIOTHVAP20,NHBLACKAMINASIANVAP20,NHBLACKAMINNHPIVAP20,NHBLACKAMINOTHVAP20,NHBLACKASIANNHPIVAP20,NHBLACKASIANOTHVAP20,NHBLACKNHPIOTHVAP20,NHAMINASIANNHPIVAP20,NHAMINASIANOTHVAP20,NHAMINNHPIOTHVAP20,NHASIANNHPIOTHVAP20,NHWHITEBLACKAMINASIANVAP20,NHWHITEBLACKAMINNHPIVAP20,NHWHITEBLACKAMINOTHVAP20,NHWHITEBLACKASIANNHPIVAP20,NHWHITEBLACKASIANOTHVAP20,NHWHITEBLACKNHPIOTHVAP20,NHWHITEAMINASIANNHPIVAP20,NHWHITEAMINASIANOTHVAP20,NHWHITEAMINNHPIOTHVAP20,NHWHITEASIANNHPIOTHVAP20,NHBLACKAMINASIANNHPIVAP20,NHBLACKAMINASIANOTHVAP20,NHBLACKAMINNHPIOTHVAP20,NHBLACKASIANNHPIOTHVAP20,NHAMINASIANNHPIOTHVAP20,NHWHITEBLACKAMINASIANNHPIVAP20,NHWHITEBLACKAMINASIANOTHVAP20,NHWHITEBLACKAMINNHPIOTHVAP20,NHWHITEBLACKASIANNHPIOTHVAP20,NHWHITEAMINASIANNHPIOTHVAP20,NHBLACKAMINASIANNHPIOTHVAP20,NHWHITEBLACKAMINASIANNHPIOTHVAP20,NHVAP20
0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,16,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

### First, if we take both H and NH for each SINGLE race, how much of the population does that cover?

In [80]:
races = ['WHITE', 'BLACK', 'AMIN', 'ASIAN', 'NHPI', 'OTH']

In [81]:
single_race_cols = ['H'+race+'VAP20' for race in races] + ['NH'+race+'VAP20' for race in races]

In [82]:
vap_df['VAP20'].sum()

3370652

In [83]:
vap_df[single_race_cols].sum(axis=1).sum()

3076781

In [84]:
vap_df[single_race_cols].sum(axis=1).sum()/vap_df['VAP20'].sum()

0.9128147907289154

### What if we add White + one other race?

In [85]:
white_double_race_cols = ['HWHITE'+race+'VAP20' for race in races[1:]] + ['NHWHITE'+race+'VAP20' for race in races[1:]]

In [86]:
white_double_race_cols

['HWHITEBLACKVAP20',
 'HWHITEAMINVAP20',
 'HWHITEASIANVAP20',
 'HWHITENHPIVAP20',
 'HWHITEOTHVAP20',
 'NHWHITEBLACKVAP20',
 'NHWHITEAMINVAP20',
 'NHWHITEASIANVAP20',
 'NHWHITENHPIVAP20',
 'NHWHITEOTHVAP20']

In [87]:
vap_df[white_double_race_cols].sum(axis=1).sum()

263556

In [88]:
vap_df[white_double_race_cols].sum(axis=1).sum()/vap_df['VAP20'].sum()

0.0781914003581503

In [89]:
(vap_df[single_race_cols].sum(axis=1).sum() + vap_df[white_double_race_cols].sum(axis=1).sum())/vap_df['VAP20'].sum()

0.9910061910870657

### So that's almost everyone!  

### Now look at pairs of non-White races and compute the pairwise overlaps between them.

In [90]:
for i in range(1,5):
    for j in range(i+1,6):
        race1 = races[i]
        race2 = races[j]
        these_cols = [col for col in all_cols if (race1 in col) and (race2 in col)]
        this_pop = vap_df[these_cols].sum(axis=1).sum()
        this_pop_pct = this_pop/vap_df['VAP20'].sum()
        print(race1, race2, this_pop, this_pop_pct)

BLACK AMIN 6035 0.0017904547844155968
BLACK ASIAN 2620 0.0007772976860263237
BLACK NHPI 885 0.00026256047791347193
BLACK OTH 3903 0.0011579362093743287
AMIN ASIAN 3057 0.0009069461931994166
AMIN NHPI 1138 0.00033762013996105204
AMIN OTH 8478 0.002515240374859226
ASIAN NHPI 6741 0.0019999098097341407
ASIAN OTH 2231 0.0006618897471468428
NHPI OTH 718 0.00021301516739194673


### These are all really tiny!  Wonder what this looks like in a more diverse state.

### Next question: How does each race split between Hispanic/non-Hispanic?  (White is easy because we only look at White alone.)  Since the multiple non-White race overlaps are so tiny, I'm going to pretend non-White races are disjoint.

In [91]:
print("HWHITE pct:", vap_df['HWHITEVAP20'].sum()/(vap_df['HWHITEVAP20'].sum() + vap_df['NHWHITEVAP20'].sum()))

HWHITE pct: 0.031720540353042406


In [92]:
for race in races[1:]:
    h_race_cols = [col for col in hispanic_cols if race in col]
    nh_race_cols = [col for col in non_hispanic_cols if race in col]
    h_race_pop = vap_df[h_race_cols].sum(axis=1).sum()
    nh_race_pop = vap_df[nh_race_cols].sum(axis=1).sum()
    print("Race:", race, "Hispanic pop:", h_race_pop, "Non-Hispanic pop:", nh_race_pop, "Hisp pct:", h_race_pop/(h_race_pop + nh_race_pop))
    

Race: BLACK Hispanic pop: 6945 Non-Hispanic pop: 83510 Hisp pct: 0.0767785086507103
Race: AMIN Hispanic pop: 29571 Non-Hispanic pop: 107160 Hisp pct: 0.216271364942844
Race: ASIAN Hispanic pop: 5869 Non-Hispanic pop: 197916 Hisp pct: 0.028799960742939862
Race: NHPI Hispanic pop: 2107 Non-Hispanic pop: 24412 Hisp pct: 0.07945246804178137
Race: OTH Hispanic pop: 276788 Non-Hispanic pop: 60190 Hisp pct: 0.8213829982966248


### So OTH is 82% Hispanic (not surprising), AMIN is 21% Hispanic, BLACK is 7.6% Hispanic, and ASIAN/NHPI really should be combined and will probably be around 3% Hispanic.

### Now do it the other way: How is the Hispanic population divided between races?

In [93]:
for race in races:
    h_race_cols = [col for col in hispanic_cols if race in col]
    h_race_pop = vap_df[h_race_cols].sum(axis=1).sum()
    print("Race:", race, "Hispanic pop:", h_race_pop, "Pct of Hispanic pop:", h_race_pop/vap_df['HVAP20'].sum())

Race: WHITE Hispanic pop: 196137 Pct of Hispanic pop: 0.5036980538991356
Race: BLACK Hispanic pop: 6945 Pct of Hispanic pop: 0.017835405784372642
Race: AMIN Hispanic pop: 29571 Pct of Hispanic pop: 0.07594107767454043
Race: ASIAN Hispanic pop: 5869 Pct of Hispanic pop: 0.015072137731962998
Race: NHPI Hispanic pop: 2107 Pct of Hispanic pop: 0.005410971920471296
Race: OTH Hispanic pop: 276788 Pct of Hispanic pop: 0.7108173212735687


In [94]:
print(vap_df["HWHITEVAP20"].sum()/vap_df['HVAP20'].sum())
print(vap_df["HOTHVAP20"].sum()/vap_df['HVAP20'].sum())
print(vap_df["HWHITEOTHVAP20"].sum()/vap_df['HVAP20'].sum())


0.21253537548087542
0.43150639198344093
0.25057140068927614


### So LOTS of the Hispanic population is White and/or Other!  And by far the most of the rest is AMIN.

In [95]:
(vap_df["HWHITEVAP20"].sum() + vap_df["HWHITEOTHVAP20"].sum()+ vap_df["HOTHVAP20"].sum())/vap_df['HVAP20'].sum()

0.8946131681535925

### How much of the ACS Hispanic population identifies as White?

In [96]:
(tracts_df['WHITEVAP_ACS22'] - tracts_df['NHWHITEVAP_ACS22']).sum()/tracts_df['HVAP_ACS22'].sum()

0.3752749624551853

### Is there enough "Other" to account for most of the rest?

In [97]:
tracts_df['OTHVAP_ACS22'].sum()/tracts_df['HVAP_ACS22'].sum()

0.327985297424723

### Nope!  What about in 2MORE?

In [98]:
tracts_df['MULTVAP_ACS22'].sum()/tracts_df['HVAP_ACS22'].sum()

0.6687474636248258

### Yep, plenty there.

In [99]:
tracts_df['HWHITEVAP_ACS22'] = tracts_df['WHITEVAP_ACS22'] - tracts_df['NHWHITEVAP_ACS22']
tracts_df['HWHITECVAP_ACS22'] = tracts_df['WHITECVAP_ACS22'] - tracts_df['NHWHITECVAP_ACS22']


### Hispanic White citizenship rate:

In [100]:
tracts_df['HWHITECVAP_ACS22'].sum()/tracts_df['HWHITEVAP_ACS22'].sum()

0.7506664003361275

### Slightly more than the overall Hispanic citizenship rate of 71%.